# HOME CREDIT DEFAULT RISK

# Feature Engineering

> *Coming up with features is difficult, time-consuming, requires expert knowledge. ‘Applied machine learning’ is basically feature engineering.* *Andrew Ng.*




In the first notebook, we had a look at the problem statement, and all the caveats to it. We also looked at the Exploratory Data Analysis, using which we were able to draw some insights about the data. Now, we will start off with those insights gained from the EDA to come up with good sets of Features using Feature Engineering techniques. 


In [36]:
# import libraries
import pandas as pd
import numpy as np

import os
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import XGBRegressor

import warnings
warnings.filterwarnings('ignore')

import lightgbm as lgb
from lightgbm import LGBMClassifier
from lightgbm import LGBMRegressor

root = os.path.dirname(os.path.realpath("__file__"))


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Defining Utility Functions**

In [49]:
def reduce_mem_usage(data, verbose = True):
    #source: https://www.kaggle.com/gemartin/load-data-reduce-memory-usage
    '''
    This function is used to reduce the memory usage by converting the datatypes of a pandas
    DataFrame withing required limits.
    '''
    
    start_mem = data.memory_usage().sum() / 1024**2
    if verbose:
        print('-'*100)
        print('Memory usage of dataframe: {:.2f} MB'.format(start_mem))
    
    for col in data.columns:
        col_type = data[col].dtype
        
        if col_type != object:
            c_min = data[col].min()
            c_max = data[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    data[col] = data[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    data[col] = data[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    data[col] = data[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    data[col] = data[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    data[col] = data[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    data[col] = data[col].astype(np.float32)
                else:
                    data[col] = data[col].astype(np.float64)

    end_mem = data.memory_usage().sum() / 1024**2
    if verbose:
        print('Memory usage after optimization: {:.2f} MB'.format(end_mem))
        print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
        print('-'*100)
    
    return data

# 1. Data Cleaning and Preprocessing

**bureau_balance.csv and bureau.csv**

These tables contain the information related to the client's previous credits which were not with Home Credit Group, and were reported by Credit Bureau Department.

1. **bureau_balance** : 

    * First off, the bureau_balance table contains three fields, i.e. SK_ID_BUREAU, MONTHS_BALANCE and STATUS.
    
    * Since the Status follows somewhat ordinal behaviour, we start by label encoding it.

    * Next, some features are created such as weighted status, which is obtained by dividing the status by the MONTHS_BALANCE.

    * Since the data contains the timeseries, we also calculate the Exponential Weighted Moving Average of the Status and Weighted Status fields.

    * Finally, we aggregate the data over SK_ID_BUREAU, in such a way that we first aggregate it over all the data, and after that we also aggregate over the last 2 years. These 2 years would depict the more recent behaviour of the clients.

    * The aggregations performed are based on Domain Knowledge, such as mean, min, max, sum, count, etc. For EDA features, we only take the last/most recent values, as they somewhat contain the trend of all the previous values.

2. **bureau** : 

    * Firstly, we merge the bureau table with the aggregated bureau_balance table from previous step, on SK_ID_BUREAU.

    * We replace some erroneous values with NaN values. We saw some loans dating back to as long as 100 years ago. We believe they wouldn't really tell much about client's recent behaviour, so we remove them and only keep the loans in the period of 50 years.

    * We create some features by multiplications, divisions, subtractions of raw features, based on domain knowledge, such as Credit duration, annutiy to credit ratio, etc.

    * The categorical features are one-hot encoded.

    * To merge these to main table, i.e. application_train, we aggregate this table over SK_ID_CURR. We perform the aggregations again in two ways. We aggregate the credits based on the CREDIT_ACTIVE category, where we aggregate for two most popular categories separately, i.e. Active, and Closed. Later we aggregate for the remaining categories too, and merge these. We aggregated the whole data overall too. The aggregations performed are sum, mean, min, max, last, etc.

In [2]:
bureau_balance = pd.read_csv("data/bureau_balance.csv")
bureau_balance.sample(15)

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
5642251,5536930,-7,C
13121797,6767477,-17,X
7729526,6409988,-20,C
23920213,6342676,-42,0
1711879,5707092,-36,X
13526465,6388985,-83,0
9202703,6071034,-1,C
13005626,6805062,-49,C
7624461,5018492,-14,0
2289347,5011019,-57,0


In [3]:
#as we saw from EDA, bureau_balance has a variable called STATUS, which describes about the status of loan.
#it has 7 labels, we will label encode them
#so we give C as 0, and rest increasing
#also we will give X the benefit of doubt and keep it as middle value
dict_for_status = { 'C': 0, '0': 1, '1': 2, '2': 3, 'X': 4, '3': 5, '4': 6, '5': 7}
bureau_balance['STATUS'] = bureau_balance['STATUS'].map(dict_for_status)

#weighing the status with the months_balance
#converting months to positive
bureau_balance['MONTHS_BALANCE'] = np.abs(bureau_balance['MONTHS_BALANCE'])
bureau_balance['WEIGHTED_STATUS'] = bureau_balance.STATUS / (bureau_balance.MONTHS_BALANCE + 1)

#sorting the bureau_balance in ascending order of month and by the bureau SK_ID
#this is done so as to make the rolling exponential average easily for previous months till current month
bureau_balance = bureau_balance.sort_values(by=['SK_ID_BUREAU', 'MONTHS_BALANCE'], ascending=[0, 0])
#we will do exponential weighted average on the encoded status
#this is because if a person had a bad status 2 years ago, it should be given less weightage today
# we keep the latent variable alpha = 0.8 
#doing this for both weighted status and the status itself

bureau_balance['EXP_WEIGHTED_STATUS'] = bureau_balance.groupby('SK_ID_BUREAU')['WEIGHTED_STATUS'].transform(lambda x: x.ewm(alpha = 0.8).mean())
bureau_balance['EXP_ENCODED_STATUS'] = bureau_balance.groupby('SK_ID_BUREAU')['STATUS'].transform(lambda x: x.ewm(alpha = 0.8).mean())

In [8]:
bureau_balance

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS,WEIGHTED_STATUS,EXP_WEIGHTED_STATUS,EXP_ENCODED_STATUS
23544064,6842888,61,1,0.016129,1.612903e-02,1.000000e+00
23544063,6842888,60,1,0.016393,1.634937e-02,1.000000e+00
23544062,6842888,59,1,0.016667,1.660526e-02,1.000000e+00
23544061,6842888,58,2,0.033898,3.046187e-02,1.801282e+00
23544060,6842888,57,1,0.017241,1.988209e-02,1.160051e+00
...,...,...,...,...,...,...
26079757,5001709,4,0,0.000000,2.217031e-59,1.934281e-57
26079756,5001709,3,0,0.000000,4.434062e-60,3.868563e-58
26079755,5001709,2,0,0.000000,8.868123e-61,7.737125e-59
26079754,5001709,1,0,0.000000,1.773625e-61,1.547425e-59


In [9]:
# we can see that these datapoints are for 96 months i.e. 8 years.
# so we will extract the means, and exponential averages for each year separately
# first we convert month to year
bureau_balance['MONTHS_BALANCE'] = bureau_balance['MONTHS_BALANCE'] // 12

# defining our aggregations
aggregations_basic = {
    'MONTHS_BALANCE' : ['mean','max'],
    'STATUS' : ['mean','max','first'],
    'WEIGHTED_STATUS' : ['mean','sum','first'],
    'EXP_ENCODED_STATUS' : ['last'],
    'EXP_WEIGHTED_STATUS' : ['last']}

# we will be finding aggregates for each year too
aggregations_for_year = {
    'STATUS' : ['mean','max','last','first'],
    'WEIGHTED_STATUS' : ['mean','max', 'first','last'],
    'EXP_WEIGHTED_STATUS' : ['last'],
    'EXP_ENCODED_STATUS' : ['last'] }

# aggregating over whole dataset first
aggregated_bureau_balance = bureau_balance.groupby(['SK_ID_BUREAU']).agg(aggregations_basic)
aggregated_bureau_balance.columns = ['_'.join(ele).upper() for ele in aggregated_bureau_balance.columns]

# aggregating some of the features separately for latest 2 years
aggregated_bureau_years = pd.DataFrame()
for year in range(2):
    year_group = bureau_balance[bureau_balance['MONTHS_BALANCE'] == year].groupby('SK_ID_BUREAU').agg(aggregations_for_year)
    year_group.columns = ['_'.join(ele).upper() + '_YEAR_' + str(year) for ele in year_group.columns]

    if year == 0:
        aggregated_bureau_years = year_group
    else:
        aggregated_bureau_years = aggregated_bureau_years.merge(year_group, on = 'SK_ID_BUREAU', how = 'outer')

# aggregating for rest of the years
aggregated_bureau_rest_years = bureau_balance[bureau_balance.MONTHS_BALANCE > year].groupby(['SK_ID_BUREAU']).agg(aggregations_for_year)
aggregated_bureau_rest_years.columns = ['_'.join(ele).upper() + '_YEAR_REST' for ele in aggregated_bureau_rest_years.columns]

# merging with rest of the years
aggregated_bureau_years = aggregated_bureau_years.merge(aggregated_bureau_rest_years, on = 'SK_ID_BUREAU', how = 'outer')
aggregated_bureau_balance = aggregated_bureau_balance.merge(aggregated_bureau_years, on = 'SK_ID_BUREAU', how = 'inner')

# filling the missing values obtained after aggregations with 0
aggregated_bureau_balance.fillna(0, inplace = True)

In [10]:
aggregated_bureau_balance

,MONTHS_BALANCE_MEAN,MONTHS_BALANCE_MAX,STATUS_MEAN,STATUS_MAX,STATUS_FIRST,WEIGHTED_STATUS_MEAN,WEIGHTED_STATUS_SUM,WEIGHTED_STATUS_FIRST,EXP_ENCODED_STATUS_LAST,EXP_WEIGHTED_STATUS_LAST,...,STATUS_MEAN_YEAR_REST,STATUS_MAX_YEAR_REST,STATUS_LAST_YEAR_REST,STATUS_FIRST_YEAR_REST,WEIGHTED_STATUS_MEAN_YEAR_REST,WEIGHTED_STATUS_MAX_YEAR_REST,WEIGHTED_STATUS_FIRST_YEAR_REST,WEIGHTED_STATUS_LAST_YEAR_REST,EXP_WEIGHTED_STATUS_LAST_YEAR_REST,EXP_ENCODED_STATUS_LAST_YEAR_REST
SK_ID_BUREAU,,,,,,,,,,,,,,,,,,,,,
5001709,3.546392,8,0.453608,4,4,0.004936,0.478827,0.041237,3.094850e-60,3.547249e-62,...,0.602740,4.0,0.0,4.0,0.006559,0.045977,0.041237,0.00,2.114325e-45,1.844674e-43
5001710,2.963855,6,1.506024,4,4,0.022689,1.883185,0.048193,4.166370e-34,8.419081e-36,...,2.118644,4.0,0.0,4.0,0.031918,0.080000,0.048193,0.00,5.018163e-19,2.483350e-17
5001711,0.000000,0,1.750000,4,1,1.270833,5.083333,0.250000,3.403846e+00,3.297543e+00,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.000000e+00,0.000000e+00
5001712,0.368421,1,0.526316,1,1,0.037830,0.718771,0.052632,5.119999e-07,5.008223e-08,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.000000e+00,0.000000e+00
5001713,0.454545,1,4.000000,4,4,0.671057,14.763253,0.181818,4.000000e+00,3.570297e+00,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.000000e+00,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6842884,1.500000,3,1.770833,4,4,0.058626,2.814045,0.083333,4.194304e-14,1.975070e-15,...,2.875000,4.0,4.0,4.0,0.087549,0.160000,0.083333,0.16,1.584893e-01,4.000000e+00
6842885,0.500000,1,4.000000,7,1,0.933134,22.395222,0.041667,7.000000e+00,6.248019e+00,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.000000e+00,0.000000e+00
6842886,0.909091,2,0.242424,1,1,0.008268,0.272840,0.030303,3.355435e-18,1.278811e-19,...,0.888889,1.0,0.0,1.0,0.030316,0.038462,0.030303,0.00,7.622312e-03,1.999996e-01


In [11]:
aggregated_bureau_balance.to_csv("data_preprocessed/aggregated_bureau_balance.csv")
# pd.read_csv("data_preprocessed/aggregated_bureau_balance.csv", index_col = "SK_ID_BUREAU")

In [16]:
# read bureau.csv
bureau = pd.read_csv("data/bureau.csv")
bureau.sample(15)

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
1001559,166141,5453771,Closed,currency 1,-938,0,-573.0,-613.0,0.0,0,68665.50,0.000,0.0,0.0,Consumer credit,-613,NaN
34909,121022,5806140,Closed,currency 1,-673,0,-308.0,-299.0,NaN,0,274860.00,0.000,0.0,0.0,Consumer credit,-276,NaN
876066,139543,6281948,Active,currency 1,-1393,0,433.0,NaN,0.0,0,0.00,0.000,0.0,0.0,Credit card,-633,NaN
1269422,303440,5267866,Closed,currency 1,-1202,0,-836.0,-830.0,NaN,0,450000.00,0.000,NaN,0.0,Consumer credit,-830,NaN
177172,411625,5701837,Active,currency 1,-178,0,5.0,NaN,0.0,0,270000.00,45145.665,0.0,0.0,Consumer credit,-23,22500.0
1032861,408535,6768616,Closed,currency 1,-376,0,720.0,-333.0,NaN,0,639679.50,NaN,NaN,0.0,Consumer credit,-333,NaN
121844,399594,6363013,Closed,currency 1,-1230,0,-134.0,-624.0,NaN,0,562500.00,0.000,NaN,0.0,Credit card,-624,0.0
725727,166059,5025833,Closed,currency 1,-284,0,-39.0,-39.0,0.0,0,78660.00,0.000,0.0,0.0,Consumer credit,-36,0.0
994708,379701,6427528,Closed,currency 1,-434,0,-73.0,-73.0,NaN,0,107104.50,0.000,0.0,0.0,Consumer credit,-50,NaN
282716,272691,5797760,Closed,currency 1,-1718,0,12622.0,-494.0,NaN,0,90000.00,0.000,0.0,0.0,Credit card,-437,0.0


In [17]:
# merging it with aggregated bureau_balance on 'SK_ID_BUREAU'
bureau_merged = bureau.merge(aggregated_bureau_balance, on = 'SK_ID_BUREAU', how = 'left')

# from the EDA we saw some erroneous values in DAYS Fields, we will remove those
# there are some loans which ended about very long ago, around 100 years ago.
# Thus we will only keep those loans which have ended in past 50 years.
bureau_merged['DAYS_CREDIT_ENDDATE'][bureau_merged['DAYS_CREDIT_ENDDATE'] > -50*365] = np.nan
bureau_merged['DAYS_ENDDATE_FACT'][bureau_merged['DAYS_ENDDATE_FACT'] > -50*365] = np.nan
#there is also a feature which tells about the number of days ago the Credit Report Came
bureau_merged['DAYS_CREDIT_UPDATE'][bureau_merged['DAYS_CREDIT_UPDATE'] > -50*365] = np.nan

# engineering some features based on domain knowledge
bureau_merged['CREDIT_DURATION'] = np.abs(bureau_merged['DAYS_CREDIT'] - bureau_merged['DAYS_CREDIT_ENDDATE'])
bureau_merged['FLAG_OVERDUE_RECENT'] = [0 if ele == 0 else 1 for ele in bureau_merged['CREDIT_DAY_OVERDUE']]
bureau_merged['MAX_AMT_OVERDUE_DURATION_RATIO'] = bureau_merged['AMT_CREDIT_MAX_OVERDUE'] / (bureau_merged['CREDIT_DURATION'] + 0.00001)
bureau_merged['CURRENT_AMT_OVERDUE_DURATION_RATIO'] = bureau_merged['AMT_CREDIT_SUM_OVERDUE'] / (bureau_merged['CREDIT_DURATION'] + 0.00001)
bureau_merged['AMT_OVERDUE_DURATION_LEFT_RATIO'] = bureau_merged['AMT_CREDIT_SUM_OVERDUE'] / (bureau_merged['DAYS_CREDIT_ENDDATE'] + 0.00001)
bureau_merged['CNT_PROLONGED_MAX_OVERDUE_MUL'] = bureau_merged['CNT_CREDIT_PROLONG'] * bureau_merged['AMT_CREDIT_MAX_OVERDUE']
bureau_merged['CNT_PROLONGED_DURATION_RATIO'] = bureau_merged['CNT_CREDIT_PROLONG'] / (bureau_merged['CREDIT_DURATION'] + 0.00001)
bureau_merged['CURRENT_DEBT_TO_CREDIT_RATIO'] = bureau_merged['AMT_CREDIT_SUM_DEBT'] / (bureau_merged['AMT_CREDIT_SUM'] + 0.00001)
bureau_merged['CURRENT_CREDIT_DEBT_DIFF'] = bureau_merged['AMT_CREDIT_SUM'] - bureau_merged['AMT_CREDIT_SUM_DEBT']
bureau_merged['AMT_ANNUITY_CREDIT_RATIO'] = bureau_merged['AMT_ANNUITY'] / (bureau_merged['AMT_CREDIT_SUM'] + 0.00001)
bureau_merged['CREDIT_ENDDATE_UPDATE_DIFF'] = np.abs(bureau_merged['DAYS_CREDIT_UPDATE'] - bureau_merged['DAYS_CREDIT_ENDDATE'])


In [18]:
#now we will be aggregating the bureau_merged df with respect to 'SK_ID_CURR' so as to merge it with application_train later    
#firstly we will aggregate the columns based on the category of CREDIT_ACTIVE
aggregations_CREDIT_ACTIVE = {
                'DAYS_CREDIT' : ['mean','min','max','last'],
                'CREDIT_DAY_OVERDUE' : ['mean','max'],
                'DAYS_CREDIT_ENDDATE' : ['mean','max'],
                'DAYS_ENDDATE_FACT' : ['mean','min'],
                'AMT_CREDIT_MAX_OVERDUE': ['max','sum'],
                'CNT_CREDIT_PROLONG': ['max','sum'],
                'AMT_CREDIT_SUM' : ['sum','max'],
                'AMT_CREDIT_SUM_DEBT': ['sum'],
                'AMT_CREDIT_SUM_LIMIT': ['max','sum'],
                'AMT_CREDIT_SUM_OVERDUE': ['max','sum'],
                'DAYS_CREDIT_UPDATE' : ['mean','min'],
                'AMT_ANNUITY' : ['mean','sum','max'],
                'CREDIT_DURATION' : ['max','mean'],
                'FLAG_OVERDUE_RECENT': ['sum'],
                'MAX_AMT_OVERDUE_DURATION_RATIO' : ['max','sum'],
                'CURRENT_AMT_OVERDUE_DURATION_RATIO' : ['max','sum'],
                'AMT_OVERDUE_DURATION_LEFT_RATIO' : ['max', 'mean'],
                'CNT_PROLONGED_MAX_OVERDUE_MUL' : ['mean','max'],
                'CNT_PROLONGED_DURATION_RATIO' : ['mean', 'max'],
                'CURRENT_DEBT_TO_CREDIT_RATIO' : ['mean', 'min'],
                'CURRENT_CREDIT_DEBT_DIFF' : ['mean','min'],
                'AMT_ANNUITY_CREDIT_RATIO' : ['mean','max','min'],
                'CREDIT_ENDDATE_UPDATE_DIFF' : ['max','min'],
                'STATUS_MEAN' : ['mean', 'max'],
                'WEIGHTED_STATUS_MEAN' : ['mean', 'max']
                 }

#we saw from EDA that the two most common type of CREDIT ACTIVE were 'Closed' and 'Active'.
#So we will aggregate them two separately and the remaining categories separately.
categories_to_aggregate_on = ['Closed','Active']
bureau_merged_aggregated_credit = pd.DataFrame()
for i, status in enumerate(categories_to_aggregate_on):
    group = bureau_merged[bureau_merged['CREDIT_ACTIVE'] == status].groupby('SK_ID_CURR').agg(aggregations_CREDIT_ACTIVE)
    group.columns = ['_'.join(ele).upper() + '_CREDITACTIVE_' + status.upper() for ele in group.columns]

    if i==0:
        bureau_merged_aggregated_credit = group
    else:
        bureau_merged_aggregated_credit = bureau_merged_aggregated_credit.merge(group, on = 'SK_ID_CURR', how = 'outer')
#aggregating for remaining categories
bureau_merged_aggregated_credit_rest = bureau_merged[(bureau_merged['CREDIT_ACTIVE'] != 'Active') & 
                                                     (bureau_merged['CREDIT_ACTIVE'] != 'Closed')].groupby('SK_ID_CURR').agg(aggregations_CREDIT_ACTIVE)
bureau_merged_aggregated_credit_rest.columns = ['_'.join(ele).upper() + 'CREDIT_ACTIVE_REST' for ele in bureau_merged_aggregated_credit_rest.columns]

#merging with other categories
bureau_merged_aggregated_credit = bureau_merged_aggregated_credit.merge(bureau_merged_aggregated_credit_rest, on = 'SK_ID_CURR', how = 'outer')

#Encoding the categorical columns in one-hot form
currency_ohe = pd.get_dummies(bureau_merged['CREDIT_CURRENCY'], prefix = 'CURRENCY')
credit_active_ohe = pd.get_dummies(bureau_merged['CREDIT_ACTIVE'], prefix = 'CREDIT_ACTIVE')
credit_type_ohe = pd.get_dummies(bureau_merged['CREDIT_TYPE'], prefix = 'CREDIT_TYPE')

#merging the one-hot encoded columns
bureau_merged = pd.concat([bureau_merged.drop(['CREDIT_CURRENCY','CREDIT_ACTIVE','CREDIT_TYPE'], axis = 1), 
                           currency_ohe, credit_active_ohe, credit_type_ohe], axis = 1)

#aggregating the bureau_merged over all the columns
bureau_merged_aggregated = bureau_merged.drop('SK_ID_BUREAU', axis = 1).groupby('SK_ID_CURR').agg('mean')
bureau_merged_aggregated.columns = [ele + '_MEAN_OVERALL' for ele in bureau_merged_aggregated.columns]
#merging it with aggregates over categories
bureau_merged_aggregated = bureau_merged_aggregated.merge(bureau_merged_aggregated_credit, on = 'SK_ID_CURR', how = 'outer')


In [19]:
bureau_merged_aggregated 

,DAYS_CREDIT_MEAN_OVERALL,CREDIT_DAY_OVERDUE_MEAN_OVERALL,DAYS_CREDIT_ENDDATE_MEAN_OVERALL,DAYS_ENDDATE_FACT_MEAN_OVERALL,AMT_CREDIT_MAX_OVERDUE_MEAN_OVERALL,CNT_CREDIT_PROLONG_MEAN_OVERALL,AMT_CREDIT_SUM_MEAN_OVERALL,AMT_CREDIT_SUM_DEBT_MEAN_OVERALL,AMT_CREDIT_SUM_LIMIT_MEAN_OVERALL,AMT_CREDIT_SUM_OVERDUE_MEAN_OVERALL,...,CURRENT_CREDIT_DEBT_DIFF_MINCREDIT_ACTIVE_REST,AMT_ANNUITY_CREDIT_RATIO_MEANCREDIT_ACTIVE_REST,AMT_ANNUITY_CREDIT_RATIO_MAXCREDIT_ACTIVE_REST,AMT_ANNUITY_CREDIT_RATIO_MINCREDIT_ACTIVE_REST,CREDIT_ENDDATE_UPDATE_DIFF_MAXCREDIT_ACTIVE_REST,CREDIT_ENDDATE_UPDATE_DIFF_MINCREDIT_ACTIVE_REST,STATUS_MEAN_MEANCREDIT_ACTIVE_REST,STATUS_MEAN_MAXCREDIT_ACTIVE_REST,WEIGHTED_STATUS_MEAN_MEANCREDIT_ACTIVE_REST,WEIGHTED_STATUS_MEAN_MAXCREDIT_ACTIVE_REST
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,-735.000000,0.0,NaN,NaN,NaN,0.000000,2.076236e+05,85240.928571,0.00000,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100002,-874.000000,0.0,NaN,NaN,1681.029,0.000000,1.081319e+05,49156.200000,7997.14125,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100003,-1400.750000,0.0,NaN,NaN,0.000,0.000000,2.543501e+05,0.000000,202500.00000,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100004,-867.000000,0.0,NaN,NaN,0.000,0.000000,9.451890e+04,0.000000,0.00000,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100005,-190.666667,0.0,NaN,NaN,0.000,0.000000,2.190420e+05,189469.500000,0.00000,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456249,-1667.076923,0.0,NaN,NaN,6147.000,0.000000,2.841430e+05,16307.100000,0.00000,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
456250,-862.000000,0.0,NaN,NaN,0.000,0.000000,1.028820e+06,744013.365000,19422.79500,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
456253,-867.500000,0.0,NaN,NaN,NaN,0.000000,9.900000e+05,448958.250000,0.00000,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
bureau_merged_aggregated.to_csv("data_preprocessed/bureau_merged_aggregated.csv")
# pd.read_csv("data_preprocessed/bureau_merged_aggregated.csv", index_col = "SK_ID_CURR")

**previous_application.csv**

This table contains the static data related to clients and their previous credits with Home Credit Group.

1. First we start by cleaning the erroneous values. From the EDA we saw some DAYS fields with a value equal to 365243.0, they look erroneous, and so we will be replacing them with NaN values.

2. We replace the NaN values for categories with an 'XNA' category.

3. Next, we proceed to feature engineering, where we create some domain based features, such as Credit-Downpayment Ratio, Amount not approved, Credit to Goods ratio, etc.

4. We also try to predict the interest rate, inspired by one of the writeups of winners.

5. To be able to merge it with main table, we need to aggregate the rows of previous_application over SK_ID_CURR. We perform domain based aggregations, over all the previous credits for each customer, such as mean, max, min, etc. Here again we aggregate in three ways. First we perform overall aggregation, next we aggregate for first 2 applications and latest 5 applications. The First and Last are decided by the DAYS_FIRST_DUE of applications. In the end, we merge all these aggregations together.

In [35]:
previous_application = pd.read_csv("data/previous_application.csv")
previous_application.sample(15)

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
1528014,2347191,434041,Cash loans,24780.600,450000.000,512370.0,NaN,450000.000,WEDNESDAY,11,...,XNA,36.0,middle,Cash Street: middle,NaN,NaN,NaN,NaN,NaN,NaN
68709,2360217,369428,Consumer loans,10251.990,94716.000,92277.0,9472.500,94716.000,MONDAY,10,...,Consumer electronics,10.0,low_normal,POS household without interest,365243.0,-2696.0,-2426.0,-2426.0,-2420.0,1.0
789260,1703215,121069,Consumer loans,7952.175,149823.000,176323.5,0.000,149823.000,TUESDAY,16,...,Consumer electronics,24.0,low_action,POS household without interest,NaN,NaN,NaN,NaN,NaN,NaN
360725,2547191,134227,Cash loans,NaN,0.000,0.0,NaN,NaN,SATURDAY,10,...,XNA,NaN,XNA,Cash,NaN,NaN,NaN,NaN,NaN,NaN
1226126,2646984,400320,Cash loans,NaN,0.000,0.0,NaN,NaN,WEDNESDAY,15,...,XNA,NaN,XNA,Cash,NaN,NaN,NaN,NaN,NaN,NaN
1322865,1863786,268083,Cash loans,NaN,0.000,0.0,NaN,NaN,MONDAY,11,...,XNA,NaN,XNA,Cash,NaN,NaN,NaN,NaN,NaN,NaN
827082,2316599,440264,Cash loans,44502.930,450000.000,553410.0,NaN,450000.000,WEDNESDAY,19,...,XNA,18.0,middle,Cash Street: middle,NaN,NaN,NaN,NaN,NaN,NaN
460560,1882518,371832,Revolving loans,16875.000,337500.000,337500.0,NaN,337500.000,TUESDAY,10,...,XNA,0.0,XNA,Card X-Sell,NaN,NaN,NaN,NaN,NaN,NaN
1129219,2299459,100059,Consumer loans,9917.010,49593.825,44631.0,4962.825,49593.825,MONDAY,15,...,Consumer electronics,5.0,middle,POS household without interest,365243.0,-2777.0,-2657.0,-2747.0,-2739.0,0.0
363666,2810557,358049,Consumer loans,3506.445,33421.500,30046.5,3375.000,33421.500,TUESDAY,15,...,Connectivity,12.0,low_normal,POS mobile with interest,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
# first we will do some data cleaning

# sorting the applications from oldest to most recent previous loans for each user
previous_application = previous_application.sort_values(by = ['SK_ID_CURR','DAYS_FIRST_DUE'])

# in the EDA we found some erroneous values in DAYS columns, so we will replace them with NaN values
previous_application['DAYS_FIRST_DRAWING'][previous_application['DAYS_FIRST_DRAWING'] == 365243.0] = np.nan
previous_application['DAYS_FIRST_DUE'][previous_application['DAYS_FIRST_DUE'] == 365243.0] = np.nan
previous_application['DAYS_LAST_DUE_1ST_VERSION'][previous_application['DAYS_LAST_DUE_1ST_VERSION'] == 365243.0] = np.nan
previous_application['DAYS_LAST_DUE'][previous_application['DAYS_LAST_DUE'] == 365243.0] = np.nan
previous_application['DAYS_TERMINATION'][previous_application['DAYS_TERMINATION'] == 365243.0] = np.nan

# we also see abruptly large value for SELLERPLACE_AREA
previous_application['SELLERPLACE_AREA'][previous_application['SELLERPLACE_AREA'] == 4000000] = np.nan

# filling the NaN values for categories
categorical_columns = previous_application.dtypes[previous_application.dtypes == 'object'].index.tolist()
previous_application[categorical_columns] = previous_application[categorical_columns].fillna('XNA')

In [38]:
# Feature engineering

# label encoding the categorical variables
name_contract_dict = {'Approved': 0, 'Refused' : 3, 'Canceled' : 2, 'Unused offer' : 1}
previous_application['NAME_CONTRACT_STATUS'] = previous_application['NAME_CONTRACT_STATUS'].map(name_contract_dict)
yield_group_dict = {'XNA': 0, 'low_action': 1, 'low_normal': 2,'middle': 3, 'high': 4}
previous_application['NAME_YIELD_GROUP'] = previous_application['NAME_YIELD_GROUP'].map(yield_group_dict)
appl_per_contract_last_dict = {'Y':1, 'N':0}
previous_application['FLAG_LAST_APPL_PER_CONTRACT'] = previous_application['FLAG_LAST_APPL_PER_CONTRACT'].map(appl_per_contract_last_dict)
remaining_categorical_columns = previous_application.dtypes[previous_application.dtypes == 'object'].index.tolist()
for col in remaining_categorical_columns:
    encoding_dict = dict([(j,i) for i,j in enumerate(previous_application[col].unique(),1)])
    previous_application[col] = previous_application[col].map(encoding_dict)    

# engineering some features on domain knowledge
previous_application['MISSING_VALUES_TOTAL_PREV'] = previous_application.isna().sum(axis = 1)
previous_application['AMT_DECLINED'] = previous_application['AMT_APPLICATION'] - previous_application['AMT_CREDIT']
previous_application['AMT_CREDIT_GOODS_RATIO'] = previous_application['AMT_CREDIT'] / (previous_application['AMT_GOODS_PRICE'] + 0.00001)
previous_application['AMT_CREDIT_GOODS_DIFF'] = previous_application['AMT_CREDIT'] - previous_application['AMT_GOODS_PRICE']
previous_application['AMT_CREDIT_APPLICATION_RATIO'] = previous_application['AMT_APPLICATION'] / (previous_application['AMT_CREDIT'] + 0.00001)
previous_application['CREDIT_DOWNPAYMENT_RATIO'] = previous_application['AMT_DOWN_PAYMENT'] / (previous_application['AMT_CREDIT'] + 0.00001)
previous_application['GOOD_DOWNPAYMET_RATIO'] = previous_application['AMT_DOWN_PAYMENT'] / (previous_application['AMT_GOODS_PRICE'] + 0.00001)
previous_application['INTEREST_DOWNPAYMENT'] = previous_application['RATE_DOWN_PAYMENT'] * previous_application['AMT_DOWN_PAYMENT']
previous_application['INTEREST_CREDIT'] = previous_application['AMT_CREDIT'] * previous_application['RATE_INTEREST_PRIMARY']
previous_application['INTEREST_CREDIT_PRIVILEGED'] = previous_application['AMT_CREDIT'] * previous_application['RATE_INTEREST_PRIVILEGED']
previous_application['APPLICATION_AMT_TO_DECISION_RATIO'] = previous_application['AMT_APPLICATION'] / (previous_application['DAYS_DECISION'] + 0.00001) * -1
previous_application['AMT_APPLICATION_TO_SELLERPLACE_AREA'] = previous_application['AMT_APPLICATION'] / (previous_application['SELLERPLACE_AREA'] + 0.00001)
previous_application['ANNUITY'] = previous_application['AMT_CREDIT'] / (previous_application['CNT_PAYMENT'] + 0.00001)
previous_application['ANNUITY_GOODS'] = previous_application['AMT_GOODS_PRICE'] / (previous_application['CNT_PAYMENT'] + 0.00001)
previous_application['DAYS_FIRST_LAST_DUE_DIFF' ] = previous_application['DAYS_LAST_DUE'] - previous_application['DAYS_FIRST_DUE']
previous_application['AMT_CREDIT_HOUR_PROCESS_START'] = previous_application['AMT_CREDIT'] * previous_application['HOUR_APPR_PROCESS_START']
previous_application['AMT_CREDIT_NFLAG_LAST_APPL_DAY'] = previous_application['AMT_CREDIT'] * previous_application['NFLAG_LAST_APPL_IN_DAY']
previous_application['AMT_CREDIT_YIELD_GROUP'] = previous_application['AMT_CREDIT'] * previous_application['NAME_YIELD_GROUP']

#https://www.kaggle.com/c/home-credit-default-risk/discussion/64598
previous_application['AMT_INTEREST'] = previous_application['CNT_PAYMENT'] * previous_application[
                                        'AMT_ANNUITY'] - previous_application['AMT_CREDIT'] 
previous_application['INTEREST_SHARE'] = previous_application['AMT_INTEREST'] / (previous_application[
                                                                                        'AMT_CREDIT'] + 0.00001)
previous_application['INTEREST_RATE'] = 2 * 12 * previous_application['AMT_INTEREST'] / (previous_application[
                                    'AMT_CREDIT'] * (previous_application['CNT_PAYMENT'] + 1))


In [40]:
# finally the aggregations

# definning the aggregations
aggregations_for_previous_application = {
    'MISSING_VALUES_TOTAL_PREV' : ['sum'],
    'NAME_CONTRACT_TYPE' : ['mean','last'],
    'AMT_ANNUITY' : ['mean','sum','max'],
    'AMT_APPLICATION' : ['mean','max','sum'],
    'AMT_CREDIT' : ['mean','max','sum'],
    'AMT_DOWN_PAYMENT' : ['mean','max','sum'],
    'AMT_GOODS_PRICE' : ['mean','max','sum'],
    'WEEKDAY_APPR_PROCESS_START' : ['mean','max','min'],
    'HOUR_APPR_PROCESS_START' : ['mean','max','min'],
    'FLAG_LAST_APPL_PER_CONTRACT' : ['mean','sum'],
    'NFLAG_LAST_APPL_IN_DAY' : ['mean','sum'],
    'RATE_DOWN_PAYMENT' : ['mean','max'],
    'RATE_INTEREST_PRIMARY' : ['mean','max'],
    'RATE_INTEREST_PRIVILEGED' : ['mean','max'],
    'NAME_CASH_LOAN_PURPOSE' : ['mean','last'],
    'NAME_CONTRACT_STATUS' : ['mean','max','last'],
    'DAYS_DECISION' : ['mean','max','min'],
    'NAME_PAYMENT_TYPE' : ['mean', 'last'],
    'CODE_REJECT_REASON' : ['mean','last'],
    'NAME_TYPE_SUITE' : ['mean','last'],
    'NAME_CLIENT_TYPE' : ['mean','last'],
    'NAME_GOODS_CATEGORY' : ['mean','last'],
    'NAME_PORTFOLIO' : ['mean','last'],
    'NAME_PRODUCT_TYPE' : ['mean','last'],
    'CHANNEL_TYPE' : ['mean','last'],
    'SELLERPLACE_AREA' : ['mean','max','min'],
    'NAME_SELLER_INDUSTRY' : ['mean','last'],
    'CNT_PAYMENT' : ['sum','mean','max'],
    'NAME_YIELD_GROUP' : ['mean','last'],
    'PRODUCT_COMBINATION' : ['mean', 'last'],
    'DAYS_FIRST_DRAWING' : ['mean','max'],
    'DAYS_FIRST_DUE' : ['mean','max'],
    'DAYS_LAST_DUE_1ST_VERSION' : ['mean'],
    'DAYS_LAST_DUE' : ['mean'],
    'DAYS_TERMINATION' : ['mean','max'],
    'NFLAG_INSURED_ON_APPROVAL' : ['sum'],
    'AMT_DECLINED' : ['mean','max','sum'],
    'AMT_CREDIT_GOODS_RATIO' : ['mean', 'max', 'min'],
    'AMT_CREDIT_GOODS_DIFF' : ['sum','mean','max', 'min'],
    'AMT_CREDIT_APPLICATION_RATIO' : ['mean','min'],
    'CREDIT_DOWNPAYMENT_RATIO' : ['mean','max'],
    'GOOD_DOWNPAYMET_RATIO' : ['mean','max'],
    'INTEREST_DOWNPAYMENT' : ['mean','sum','max'],
    'INTEREST_CREDIT' : ['mean','sum','max'],
    'INTEREST_CREDIT_PRIVILEGED' : ['mean','sum','max'],
    'APPLICATION_AMT_TO_DECISION_RATIO' : ['mean','min'],
    'AMT_APPLICATION_TO_SELLERPLACE_AREA' : ['mean','max'],
    'ANNUITY' : ['mean','sum','max'],
    'ANNUITY_GOODS' : ['mean','sum','max'],
    'DAYS_FIRST_LAST_DUE_DIFF' : ['mean','max'],
    'AMT_CREDIT_HOUR_PROCESS_START' : ['mean','sum'],
    'AMT_CREDIT_NFLAG_LAST_APPL_DAY' : ['mean','max'],
    'AMT_CREDIT_YIELD_GROUP' : ['mean','sum','min'],
    'AMT_INTEREST' : ['mean','sum','max','min'],
    'INTEREST_SHARE' : ['mean','max','min'],
    'INTEREST_RATE' : ['mean','max','min']
}

# grouping the previous applications over SK_ID_CURR while only taking the latest 5 applications
group_last_3 = previous_application.groupby('SK_ID_CURR').tail(5).groupby('SK_ID_CURR').agg(aggregations_for_previous_application)
group_last_3.columns = ['_'.join(ele).upper() + '_LAST_5' for ele in group_last_3.columns]

# grouping the previous applications over SK_ID_CURR while only taking the first 2 applications
group_first_3 = previous_application.groupby('SK_ID_CURR').head(2).groupby('SK_ID_CURR').agg(aggregations_for_previous_application)
group_first_3.columns = ['_'.join(ele).upper() + '_FIRST_2' for ele in group_first_3.columns]

# grouping the previous applications over SK_ID_CURR while taking all the applications into consideration
group_all = previous_application.groupby('SK_ID_CURR').agg(aggregations_for_previous_application)
group_all.columns = ['_'.join(ele).upper() + '_ALL' for ele in group_all.columns]

# merging all the applications
previous_application_aggregated = group_last_3.merge(group_first_3, on = 'SK_ID_CURR', how = 'outer')
previous_application_aggregated = previous_application_aggregated.merge(group_all, on = 'SK_ID_CURR', how = 'outer')

In [41]:
previous_application_aggregated

,MISSING_VALUES_TOTAL_PREV_SUM_LAST_5,NAME_CONTRACT_TYPE_MEAN_LAST_5,NAME_CONTRACT_TYPE_LAST_LAST_5,AMT_ANNUITY_MEAN_LAST_5,AMT_ANNUITY_SUM_LAST_5,AMT_ANNUITY_MAX_LAST_5,AMT_APPLICATION_MEAN_LAST_5,AMT_APPLICATION_MAX_LAST_5,AMT_APPLICATION_SUM_LAST_5,AMT_CREDIT_MEAN_LAST_5,...,AMT_INTEREST_MEAN_ALL,AMT_INTEREST_SUM_ALL,AMT_INTEREST_MAX_ALL,AMT_INTEREST_MIN_ALL,INTEREST_SHARE_MEAN_ALL,INTEREST_SHARE_MAX_ALL,INTEREST_SHARE_MIN_ALL,INTEREST_RATE_MEAN_ALL,INTEREST_RATE_MAX_ALL,INTEREST_RATE_MIN_ALL
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,3,1.000000,1,3951.0000,3951.000,3951.000,24835.50,24835.5,24835.5,23787.00,...,7821.00000,7821.00,7821.00,7821.00,0.328793,0.328793,0.328793,0.876781,0.876781,0.876781
100002,3,1.000000,1,9251.7750,9251.775,9251.775,179055.00,179055.0,179055.0,179055.00,...,42987.60000,42987.60,42987.60,42987.60,0.240080,0.240080,0.240080,0.230477,0.230477,0.230477
100003,11,1.333333,2,56553.9900,169661.970,98356.995,435436.50,900000.0,1306309.5,484191.00,...,65321.55000,195964.65,144401.94,12794.22,0.146201,0.188002,0.111200,0.328564,0.381257,0.257354
100004,3,1.000000,1,5357.2500,5357.250,5357.250,24282.00,24282.0,24282.0,20106.00,...,1323.00000,1323.00,1323.00,1323.00,0.065801,0.065801,0.065801,0.315846,0.315846,0.315846
100005,16,1.500000,2,4813.2000,4813.200,4813.200,22308.75,44617.5,44617.5,20076.75,...,17604.90000,17604.90,17604.90,17604.90,0.438440,0.438440,0.438440,0.809428,0.809428,0.809428
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456251,3,1.000000,1,6605.9100,6605.910,6605.910,40455.00,40455.0,40455.0,40455.00,...,12392.28000,12392.28,12392.28,12392.28,0.306323,0.306323,0.306323,0.816860,0.816860,0.816860
456252,3,1.000000,1,10074.4650,10074.465,10074.465,57595.50,57595.5,57595.5,56821.50,...,3625.29000,3625.29,3625.29,3625.29,0.063801,0.063801,0.063801,0.218748,0.218748,0.218748
456253,6,1.000000,1,4770.4050,9540.810,5567.715,24162.75,28912.5,48325.5,20625.75,...,4023.58500,8047.17,6100.29,1946.88,0.181506,0.223405,0.139606,0.718035,0.765959,0.670110


In [42]:
previous_application_aggregated.to_csv("data_preprocessed/previous_application_aggregated.csv")
# pd.read_csv("data_preprocessed/previous_application_aggregated.csv", index_col = "SK_ID_CURR")

**installments_payments.csv**

This table contains the details about each installment of client's previous credits with Home Credit Group.

1. We start by sorting the data first by SK_ID_CURR and SK_ID_PREV, and then by NUM_INSTALMENT_NUMBER. This brings the latest installments in the end.

2. We create some features, such as the number of days the payment was delayed, the difference in amount of payment required vs paid, etc.

3. Next we aggregate these rows over SK_ID_PREV, such that each client's previous loan gets one row. These aggregations are done in three ways, first overall aggregations, second we aggregate only those installments which were in the last 365 days, and lastly, we aggregate the first 5 installments of every loan. This will help us to capture the starting behaviour, the latest behaviour and the overall behaviour of the client's installments payments.

4. Now to merge this table with main table, we aggregate the data over SK_ID_CURR.

In [44]:
# load the installments_payments.csv DataFrame.
installments_payments = pd.read_csv("data/installments_payments.csv")
installments_payments.sample(15)

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
3801182,2229023,231981,1.0,14,-162.0,-176.0,10030.680,10030.680
3560669,2793662,179116,1.0,5,-2379.0,-2382.0,11968.785,11883.825
7703712,1234914,398226,2.0,6,-1059.0,-1063.0,173683.215,173683.215
10704538,1162461,325748,0.0,67,-1579.0,-1579.0,187.965,187.965
9203178,1936994,348915,1.0,1,-2186.0,-2192.0,9532.305,9532.305
12647154,2605547,432931,0.0,40,-1573.0,-1573.0,9000.000,9000.000
6493845,2581652,269898,0.0,7,-2528.0,-2538.0,3375.000,3375.000
443666,1182870,134908,1.0,6,-961.0,-962.0,18324.765,18324.765
1507795,1674932,123485,1.0,2,-2131.0,-2130.0,15997.140,15997.140
4466256,2683620,266948,0.0,20,-339.0,-340.0,43688.475,43688.475


In [45]:
# data cleaning and feature engineering

# sorting by SK_ID_PREV and NUM_INSTALMENT_NUMBER
installments_payments = installments_payments.sort_values(by = ['SK_ID_CURR','SK_ID_PREV','NUM_INSTALMENT_NUMBER'], ascending = True)

# getting the total NaN values in the table
installments_payments['MISSING_VALS_TOTAL_INSTAL'] = installments_payments.isna().sum(axis = 1)

# engineering new features based on some domain based polynomial operations
installments_payments['DAYS_PAYMENT_RATIO'] = installments_payments['DAYS_INSTALMENT'] / (installments_payments['DAYS_ENTRY_PAYMENT'] + 0.00001)
installments_payments['DAYS_PAYMENT_DIFF'] = installments_payments['DAYS_INSTALMENT'] - installments_payments['DAYS_ENTRY_PAYMENT']
installments_payments['AMT_PAYMENT_RATIO'] = installments_payments['AMT_PAYMENT'] / (installments_payments['AMT_INSTALMENT'] + 0.00001)
installments_payments['AMT_PAYMENT_DIFF'] = installments_payments['AMT_INSTALMENT'] - installments_payments['AMT_PAYMENT']
installments_payments['EXP_DAYS_PAYMENT_RATIO'] = installments_payments['DAYS_PAYMENT_RATIO'].transform(lambda x: x.ewm(alpha = 0.5).mean())
installments_payments['EXP_DAYS_PAYMENT_DIFF'] = installments_payments['DAYS_PAYMENT_DIFF'].transform(lambda x: x.ewm(alpha = 0.5).mean())
installments_payments['EXP_AMT_PAYMENT_RATIO'] = installments_payments['AMT_PAYMENT_RATIO'].transform(lambda x: x.ewm(alpha = 0.5).mean())
installments_payments['EXP_AMT_PAYMENT_DIFF'] = installments_payments['AMT_PAYMENT_DIFF'].transform(lambda x: x.ewm(alpha = 0.5).mean())

In [46]:
# aggregating the data over SK_ID_PREV, i.e. for each previous loan
overall_aggregations = {
    'MISSING_VALS_TOTAL_INSTAL' : ['sum'],
    'NUM_INSTALMENT_VERSION' : ['mean','sum'],
    'NUM_INSTALMENT_NUMBER' : ['max'],
    'DAYS_INSTALMENT' : ['max','min'],
    'DAYS_ENTRY_PAYMENT' : ['max','min'],
    'AMT_INSTALMENT' : ['mean', 'sum', 'max'],
    'AMT_PAYMENT' : ['mean', 'sum', 'max'],
    'DAYS_PAYMENT_RATIO' : ['mean', 'min','max'],
    'DAYS_PAYMENT_DIFF' : ['mean','min','max'],
    'AMT_PAYMENT_RATIO' : ['mean','min','max'],
    'AMT_PAYMENT_DIFF' : ['mean','min','max'],
    'EXP_DAYS_PAYMENT_RATIO' : ['last'],
    'EXP_DAYS_PAYMENT_DIFF' : ['last'],
    'EXP_AMT_PAYMENT_RATIO' : ['last'],
    'EXP_AMT_PAYMENT_DIFF' : ['last']
}
limited_period_aggregations = {
    'NUM_INSTALMENT_VERSION' : ['mean','sum'],
    'AMT_INSTALMENT' : ['mean', 'sum', 'max'],
    'AMT_PAYMENT' : ['mean', 'sum', 'max'],
    'DAYS_PAYMENT_RATIO' : ['mean', 'min','max'],
    'DAYS_PAYMENT_DIFF' : ['mean','min','max'],
    'AMT_PAYMENT_RATIO' : ['mean','min','max'],
    'AMT_PAYMENT_DIFF' : ['mean','min','max'],
    'EXP_DAYS_PAYMENT_RATIO' : ['last'],
    'EXP_DAYS_PAYMENT_DIFF' : ['last'],
    'EXP_AMT_PAYMENT_RATIO' : ['last'],
    'EXP_AMT_PAYMENT_DIFF' : ['last']
}

# aggregating installments_payments over SK_ID_PREV for last 1 year installments
group_last_1_year = installments_payments[installments_payments['DAYS_INSTALMENT'] > -365].groupby('SK_ID_PREV').agg(limited_period_aggregations)
group_last_1_year.columns = ['_'.join(ele).upper() + '_LAST_1_YEAR' for ele in group_last_1_year.columns]

# aggregating installments_payments over SK_ID_PREV for first 5 installments
group_first_5_instalments = installments_payments.groupby('SK_ID_PREV', as_index = False).head(5).groupby('SK_ID_PREV').agg(limited_period_aggregations)
group_first_5_instalments.columns = ['_'.join(ele).upper() + '_FIRST_5_INSTALLMENTS' for ele in group_first_5_instalments.columns]

# overall aggregation of installments_payments over SK_ID_PREV
group_overall = installments_payments.groupby(['SK_ID_PREV','SK_ID_CURR'], as_index = False).agg(overall_aggregations)
group_overall.columns = ['_'.join(ele).upper() for ele in group_overall.columns]
group_overall.rename(columns = {'SK_ID_PREV_': 'SK_ID_PREV','SK_ID_CURR_' : 'SK_ID_CURR'}, inplace = True)

# merging all of the above aggregations together
installments_payments_agg_prev = group_overall.merge(group_last_1_year, on = 'SK_ID_PREV', how = 'outer')
installments_payments_agg_prev = installments_payments_agg_prev.merge(group_first_5_instalments, on = 'SK_ID_PREV', how = 'outer')

In [47]:
# aggregating the data over over SK_ID_CURR
main_features_aggregations = {
    'MISSING_VALS_TOTAL_INSTAL_SUM' : ['sum'],
    'NUM_INSTALMENT_VERSION_MEAN' : ['mean'],
    'NUM_INSTALMENT_VERSION_SUM' : ['mean'],
    'NUM_INSTALMENT_NUMBER_MAX' : ['mean','sum','max'],
    'AMT_INSTALMENT_MEAN' : ['mean','sum','max'],
    'AMT_INSTALMENT_SUM' : ['mean','sum','max'],
    'AMT_INSTALMENT_MAX' : ['mean'],
    'AMT_PAYMENT_MEAN' : ['mean','sum','max'],
    'AMT_PAYMENT_SUM' : ['mean','sum','max'],
    'AMT_PAYMENT_MAX' : ['mean'],
    'DAYS_PAYMENT_RATIO_MEAN' : ['mean','min','max'],
    'DAYS_PAYMENT_RATIO_MIN' : ['mean','min'],
    'DAYS_PAYMENT_RATIO_MAX' : ['mean','max'],
    'DAYS_PAYMENT_DIFF_MEAN' : ['mean','min','max'],
    'DAYS_PAYMENT_DIFF_MIN' : ['mean','min'],
    'DAYS_PAYMENT_DIFF_MAX' : ['mean','max'],
    'AMT_PAYMENT_RATIO_MEAN' : ['mean', 'min','max'],
    'AMT_PAYMENT_RATIO_MIN' : ['mean','min'],
    'AMT_PAYMENT_RATIO_MAX' : ['mean','max'],
    'AMT_PAYMENT_DIFF_MEAN' : ['mean','min','max'],
    'AMT_PAYMENT_DIFF_MIN' : ['mean','min'],
    'AMT_PAYMENT_DIFF_MAX' : ['mean','max'],
    'EXP_DAYS_PAYMENT_RATIO_LAST' : ['mean'],
    'EXP_DAYS_PAYMENT_DIFF_LAST' : ['mean'],
    'EXP_AMT_PAYMENT_RATIO_LAST' : ['mean'],
    'EXP_AMT_PAYMENT_DIFF_LAST' : ['mean']
}

grouped_main_features = installments_payments_agg_prev.groupby('SK_ID_CURR').agg(main_features_aggregations)
grouped_main_features.columns = ['_'.join(ele).upper() for ele in grouped_main_features.columns]

#group remaining ones
grouped_remaining_features = installments_payments_agg_prev.iloc[:,[1] + list(range(31,len(installments_payments_agg_prev.columns)))].groupby('SK_ID_CURR').mean()

installments_payments_aggregated = grouped_main_features.merge(grouped_remaining_features, on = 'SK_ID_CURR', how = 'inner')

In [48]:
installments_payments_aggregated

,MISSING_VALS_TOTAL_INSTAL_SUM_SUM,NUM_INSTALMENT_VERSION_MEAN_MEAN,NUM_INSTALMENT_VERSION_SUM_MEAN,NUM_INSTALMENT_NUMBER_MAX_MEAN,NUM_INSTALMENT_NUMBER_MAX_SUM,NUM_INSTALMENT_NUMBER_MAX_MAX,AMT_INSTALMENT_MEAN_MEAN,AMT_INSTALMENT_MEAN_SUM,AMT_INSTALMENT_MEAN_MAX,AMT_INSTALMENT_SUM_MEAN,...,AMT_PAYMENT_RATIO_MEAN_FIRST_5_INSTALLMENTS,AMT_PAYMENT_RATIO_MIN_FIRST_5_INSTALLMENTS,AMT_PAYMENT_RATIO_MAX_FIRST_5_INSTALLMENTS,AMT_PAYMENT_DIFF_MEAN_FIRST_5_INSTALLMENTS,AMT_PAYMENT_DIFF_MIN_FIRST_5_INSTALLMENTS,AMT_PAYMENT_DIFF_MAX_FIRST_5_INSTALLMENTS,EXP_DAYS_PAYMENT_RATIO_LAST_FIRST_5_INSTALLMENTS,EXP_DAYS_PAYMENT_DIFF_LAST_FIRST_5_INSTALLMENTS,EXP_AMT_PAYMENT_RATIO_LAST_FIRST_5_INSTALLMENTS,EXP_AMT_PAYMENT_DIFF_LAST_FIRST_5_INSTALLMENTS
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,0,1.125000,4.000000,4.000000,8,4,5647.200000,11294.400000,7312.725000,20597.9625,...,1.000000,1.000000,1.000000,0.0000,0.0000,0.0000,0.996151,5.844619,1.000000,0.000000e+00
100002,0,1.052632,20.000000,19.000000,19,19,11559.247105,11559.247105,11559.247105,219625.6950,...,1.000000,1.000000,1.000000,0.0000,0.0000,0.0000,0.954027,22.591941,1.000000,0.000000e+00
100003,0,1.047619,8.666667,8.333333,25,12,78558.479286,235675.437857,164425.332857,539621.5500,...,1.000000,1.000000,1.000000,0.0000,0.0000,0.0000,0.987843,8.158240,1.000000,0.000000e+00
100004,0,1.333333,4.000000,3.000000,3,3,7096.155000,7096.155000,7096.155000,21288.4650,...,1.000000,1.000000,1.000000,0.0000,0.0000,0.0000,0.990915,6.728890,1.000000,0.000000e+00
100005,0,1.111111,10.000000,9.000000,9,9,6240.205000,6240.205000,6240.205000,56161.8450,...,1.000000,1.000000,1.000000,0.0000,0.0000,0.0000,0.977839,14.960278,1.000000,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456251,0,1.142857,8.000000,7.000000,7,7,7492.924286,7492.924286,7492.924286,52450.4700,...,1.000000,1.000000,1.000000,0.0000,0.0000,0.0000,0.717343,43.919551,1.000000,2.400011e-06
456252,0,1.000000,6.000000,6.000000,6,6,10069.867500,10069.867500,10069.867500,60419.2050,...,1.000000,1.000000,1.000000,0.0000,0.0000,0.0000,0.990615,2.475934,1.000000,1.875009e-08
456253,0,1.000000,4.666667,7.333333,22,12,4104.814000,12314.442000,5568.960000,20531.9700,...,0.933333,0.668955,1.000000,264.8730,0.0000,1315.2750,0.993455,13.436857,0.956654,1.722162e+02


In [49]:
installments_payments_aggregated.to_csv("data_preprocessed/installments_payments_aggregated.csv")
# pd.read_csv("data_preprocessed/installments_payments_aggregated.csv", index_col = "SK_ID_CURR")

**POS_CASH_balance.csv**

This table contains the Monthly Balance Snapshots of previous Point of Sales and Cash Loans that the applicant had with Home Credit Group. The table contains columns like the status of contract, the number of installments left, etc.

1. Similar to bureau_balance table, this table also has time based features. So we start off by computing the EDAs on CNT_INSTALMENT and CNT_INSTALMENT_FUTURE features.

2. We create some domain based features next.

3. We then aggregate the data over SK_ID_PREV. For this aggregation, we do it in 3 ways. Firstly we aggregate the whole data over SK_ID_PREV. We also aggregate the data for last 2 years separately and rest of the years separately. Finally, we also aggregate the data different Contract types, i.e. Active and Completed.

4. Next, we aggregate the data over SK_ID_CURR, for it to be merged with main table.

In [2]:
# load the POS_CASH_balance.csv DataFrame.
pos_cash = pd.read_csv("data/POS_CASH_balance.csv")
pos_cash.sample(15)

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
1763127,2571466,155706,-28,12.0,10.0,Active,0,0
4130502,1890791,434764,-36,10.0,4.0,Active,0,0
8396276,2174936,176977,-17,12.0,3.0,Active,0,0
1969270,1666366,448698,-8,24.0,24.0,Active,0,0
9327232,2356048,207909,-74,24.0,21.0,Active,0,0
8408844,2118223,126158,-35,12.0,7.0,Active,0,0
3896467,2182861,370088,-30,10.0,10.0,Active,0,0
4824209,1460831,310203,-30,18.0,0.0,Completed,0,0
9304539,2030173,288402,-52,9.0,1.0,Active,0,0
3918967,2210348,366150,-30,24.0,0.0,Active,0,0


In [3]:
# data_preprocessing and feature engineering

# making the MONTHS_BALANCE Positive
pos_cash['MONTHS_BALANCE'] = np.abs(pos_cash['MONTHS_BALANCE'])

# sorting the DataFrame according to the month of status from oldest to latest, for rolling computations
pos_cash = pos_cash.sort_values(by=['SK_ID_PREV', 'MONTHS_BALANCE'], ascending=False)

# computing Exponential Moving Average for some features based on MONTHS_BALANCE
columns_for_ema = ['CNT_INSTALMENT', 'CNT_INSTALMENT_FUTURE']
exp_columns = ['EXP_'+ele for ele in columns_for_ema]
pos_cash[exp_columns] = pos_cash.groupby('SK_ID_PREV')[columns_for_ema].transform(lambda x: x.ewm(alpha = 0.6).mean())

# creating new features based on Domain Knowledge
pos_cash['SK_DPD_RATIO'] = pos_cash['SK_DPD'] / (pos_cash['SK_DPD_DEF'] + 0.00001)
pos_cash['TOTAL_TERM'] = pos_cash['CNT_INSTALMENT'] + pos_cash['CNT_INSTALMENT_FUTURE']
pos_cash['EXP_POS_TOTAL_TERM'] = pos_cash['EXP_CNT_INSTALMENT'] + pos_cash['EXP_CNT_INSTALMENT_FUTURE']


In [4]:
# aggregating over SK_ID_PREV

overall_aggregations = {
    'SK_ID_CURR' : ['first'],
    'MONTHS_BALANCE' : ['max'],
    'CNT_INSTALMENT' : ['mean', 'max','min'],
    'CNT_INSTALMENT_FUTURE' : ['mean','max','min'],
    'SK_DPD' : ['max','sum'],
    'SK_DPD_DEF' : ['max','sum'],
    'EXP_CNT_INSTALMENT' : ['last'],
    'EXP_CNT_INSTALMENT_FUTURE' : ['last'],
    'SK_DPD_RATIO' : ['mean','max'],
    'TOTAL_TERM' : ['mean','max','last'],
    'EXP_POS_TOTAL_TERM' : ['mean'] 
}

aggregations_for_year = {
    'CNT_INSTALMENT' : ['mean', 'max','min'],
    'CNT_INSTALMENT_FUTURE' : ['mean','max','min'],
    'SK_DPD' : ['max','sum'],
    'SK_DPD_DEF' : ['max','sum'],
    'EXP_CNT_INSTALMENT' : ['last'],
    'EXP_CNT_INSTALMENT_FUTURE' : ['last'],
    'SK_DPD_RATIO' : ['mean','max'],
    'TOTAL_TERM' : ['mean','max'],
    'EXP_POS_TOTAL_TERM' : ['last'] 
}

aggregations_for_categories = {
    'CNT_INSTALMENT' : ['mean', 'max','min'],
    'CNT_INSTALMENT_FUTURE' : ['mean','max','min'],
    'SK_DPD' : ['max','sum'],
    'SK_DPD_DEF' : ['max','sum'],
    'EXP_CNT_INSTALMENT' : ['last'],
    'EXP_CNT_INSTALMENT_FUTURE' : ['last'],
    'SK_DPD_RATIO' : ['mean','max'],
    'TOTAL_TERM' : ['mean','max'],
    'EXP_POS_TOTAL_TERM' : ['last']
}

# performing overall aggregations over SK_ID_PREV
pos_cash_aggregated_overall = pos_cash.groupby('SK_ID_PREV').agg(overall_aggregations)
pos_cash_aggregated_overall.columns = ['_'.join(ele).upper() for ele in pos_cash_aggregated_overall.columns]
pos_cash_aggregated_overall.rename(columns = {'SK_ID_CURR_FIRST': 'SK_ID_CURR'}, inplace = True)

# yearwise aggregations
pos_cash['YEAR_BALANCE'] = pos_cash['MONTHS_BALANCE'] //12

# aggregating over SK_ID_PREV for each last 2 years
pos_cash_aggregated_year = pd.DataFrame()
for year in range(2):
    group = pos_cash[pos_cash['YEAR_BALANCE'] == year].groupby('SK_ID_PREV').agg(aggregations_for_year)
    group.columns = ['_'.join(ele).upper() + '_YEAR_' + str(year) for ele in group.columns]
    if year == 0:
        pos_cash_aggregated_year = group
    else:
        pos_cash_aggregated_year = pos_cash_aggregated_year.merge(group, on = 'SK_ID_PREV', how = 'outer')

# aggregating over SK_ID_PREV for rest of the years
pos_cash_aggregated_rest_years = pos_cash[pos_cash['YEAR_BALANCE'] >= 2].groupby('SK_ID_PREV').agg(aggregations_for_year)
pos_cash_aggregated_rest_years.columns = ['_'.join(ele).upper() + '_YEAR_REST' for ele in pos_cash_aggregated_rest_years.columns]

# merging all the years aggregations
pos_cash_aggregated_year = pos_cash_aggregated_year.merge(pos_cash_aggregated_rest_years, on = 'SK_ID_PREV', how = 'outer')
pos_cash = pos_cash.drop(['YEAR_BALANCE'], axis = 1)

# aggregating over SK_ID_PREV for each of NAME_CONTRACT_STATUS categories
contract_type_categories = ['Active', 'Completed']
pos_cash_aggregated_contract = pd.DataFrame()
for i, contract_type in enumerate(contract_type_categories):
    group = pos_cash[pos_cash['NAME_CONTRACT_STATUS'] == contract_type].groupby('SK_ID_PREV').agg(aggregations_for_categories)
    group.columns = ['_'.join(ele).upper() + '_' + contract_type.upper() for ele in group.columns]
    if i == 0:
        pos_cash_aggregated_contract = group
    else:
        pos_cash_aggregated_contract = pos_cash_aggregated_contract.merge(group, on = 'SK_ID_PREV', how = 'outer')

pos_cash_aggregated_rest_contract = pos_cash[(pos_cash['NAME_CONTRACT_STATUS'] != 'Active') & 
                                (pos_cash['NAME_CONTRACT_STATUS'] != 'Completed')].groupby('SK_ID_PREV').agg(aggregations_for_categories)
pos_cash_aggregated_rest_contract.columns = ['_'.join(ele).upper() + '_REST' for ele in pos_cash_aggregated_rest_contract.columns]

# merging the categorical aggregations
pos_cash_aggregated_contract = pos_cash_aggregated_contract.merge(pos_cash_aggregated_rest_contract, on = 'SK_ID_PREV', how = 'outer')    

# merging all the aggregations
pos_cash_aggregated = pos_cash_aggregated_overall.merge(pos_cash_aggregated_year, on = 'SK_ID_PREV', how = 'outer')
pos_cash_aggregated = pos_cash_aggregated.merge(pos_cash_aggregated_contract, on = 'SK_ID_PREV', how = 'outer')

# onehot encoding the categorical feature NAME_CONTRACT_TYPE
name_contract_dummies = pd.get_dummies(pos_cash['NAME_CONTRACT_STATUS'], prefix='CONTRACT')
contract_names = name_contract_dummies.columns.tolist()
# concatenating one-hot encoded categories with main table
pos_cash = pd.concat([pos_cash, name_contract_dummies], axis=1)

# aggregating these over SK_ID_PREV as well
aggregated_cc_contract = pos_cash[['SK_ID_PREV'] + contract_names].groupby('SK_ID_PREV').mean()    

# merging with the final aggregations
pos_cash_aggregated = pos_cash_aggregated.merge(aggregated_cc_contract, on = 'SK_ID_PREV', how = 'outer')


In [5]:
# aggregating over SK_ID_CURR
columns_to_aggregate = pos_cash_aggregated.columns[1:]

# defining the aggregations to perform
aggregations_final = {}
for col in columns_to_aggregate:
    if 'MEAN' in col:
        aggregates = ['mean','sum','max']
    else:
        aggregates = ['mean']
    aggregations_final[col] = aggregates
pos_cash_aggregated_final = pos_cash_aggregated.groupby('SK_ID_CURR').agg(aggregations_final)
pos_cash_aggregated_final.columns = ['_'.join(ele).upper() for ele in pos_cash_aggregated_final.columns]

In [6]:
pos_cash_aggregated_final

,MONTHS_BALANCE_MAX_MEAN,CNT_INSTALMENT_MEAN_MEAN,CNT_INSTALMENT_MEAN_SUM,CNT_INSTALMENT_MEAN_MAX,CNT_INSTALMENT_MAX_MEAN,CNT_INSTALMENT_MIN_MEAN,CNT_INSTALMENT_FUTURE_MEAN_MEAN,CNT_INSTALMENT_FUTURE_MEAN_SUM,CNT_INSTALMENT_FUTURE_MEAN_MAX,CNT_INSTALMENT_FUTURE_MAX_MEAN,...,EXP_POS_TOTAL_TERM_LAST_REST_MEAN,CONTRACT_ACTIVE_MEAN,CONTRACT_AMORTIZED DEBT_MEAN,CONTRACT_APPROVED_MEAN,CONTRACT_CANCELED_MEAN,CONTRACT_COMPLETED_MEAN,CONTRACT_DEMAND_MEAN,CONTRACT_RETURNED TO THE STORE_MEAN,CONTRACT_SIGNED_MEAN,CONTRACT_XNA_MEAN
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,76.500000,4.000000,8.000000,4.000,4.000000,4.000000,1.375000,2.750000,2.000,3.0,...,NaN,0.775000,0.0,0.0,0.0,0.225000,0.0,0.0,0.000000,0.0
100002,19.000000,24.000000,24.000000,24.000,24.000000,24.000000,15.000000,15.000000,15.000,24.0,...,NaN,1.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
100003,43.333333,9.791667,29.375000,12.000,10.000000,8.333333,5.666667,17.000000,7.875,10.0,...,NaN,0.916667,0.0,0.0,0.0,0.083333,0.0,0.0,0.000000,0.0
100004,27.000000,3.750000,3.750000,3.750,4.000000,3.000000,2.250000,2.250000,2.250,4.0,...,NaN,0.750000,0.0,0.0,0.0,0.250000,0.0,0.0,0.000000,0.0
100005,25.000000,11.700000,11.700000,11.700,12.000000,9.000000,7.200000,7.200000,7.200,12.0,...,NaN,0.818182,0.0,0.0,0.0,0.090909,0.0,0.0,0.090909,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456251,9.000000,7.875000,7.875000,7.875,8.000000,7.000000,4.375000,4.375000,4.375,8.0,...,NaN,0.777778,0.0,0.0,0.0,0.111111,0.0,0.0,0.111111,0.0
456252,82.000000,6.000000,6.000000,6.000,6.000000,6.000000,3.000000,3.000000,3.000,6.0,...,NaN,0.857143,0.0,0.0,0.0,0.142857,0.0,0.0,0.000000,0.0
456253,84.333333,7.333333,22.000000,12.000,7.333333,7.333333,1.805556,5.416667,3.000,4.0,...,NaN,0.861111,0.0,0.0,0.0,0.138889,0.0,0.0,0.000000,0.0


In [7]:
pos_cash_aggregated_final.to_csv("data_preprocessed/pos_cash_aggregated_final.csv")
# pd.read_csv("data_preprocessed/pos_cash_aggregated_final.csv", index_col = "SK_ID_CURR")

**credit_card_balance.csv**

This table contains information about the previous credit cards that the client had with Home Credit Group.

1. We start off with removing an erroneous value, and then we proceed to feature engineering.

2. We create some domain based features such as total drawings, number of drawings, balance to limit ratio, payment done to minimum payment required difference, etc.

3. This table also contains all these data monthwise, so we calculate the EDAs for some of the features of this table too.

4. For aggregations, we first aggregate over SK_ID_PREV. Here we aggregate on three bases. Firstly, we do overall aggregations. We also do aggregations for last 2 years separately and the rest of the years. Finally we aggregate over SK_ID_PREV for categorical variable NAME_CONTRACT_TYPE.

5. For aggregation over SK_ID_CURR, we saw from the EDA that most of the current clients just had 1 credit card previously, so we do simple mean aggregations over SK_ID_CURR.

In [8]:
# load the credit_card_balance.csv DataFrame.
cc_balance = pd.read_csv("data/credit_card_balance.csv")
cc_balance.sample(15)

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
2874656,2341416,349216,-53,769.500,67500,0.0,0.0,0.0,0.0,692.595,...,769.500,769.500,0.0,0,0.0,0.0,37.0,Active,1,0
3708631,2537785,453968,-10,0.000,450000,0.0,0.0,0.0,0.0,0.000,...,0.000,0.000,0.0,0,0.0,0.0,17.0,Active,0,0
2875939,1986044,131664,-25,67.365,135000,0.0,0.0,0.0,0.0,0.000,...,67.365,67.365,0.0,0,0.0,0.0,54.0,Active,1066,0
775346,2567671,257993,-2,82058.040,135000,0.0,0.0,0.0,0.0,6750.000,...,82058.040,82058.040,0.0,0,0.0,0.0,90.0,Active,0,0
1542569,1855530,217602,-17,0.000,45000,0.0,0.0,0.0,0.0,0.000,...,0.000,0.000,0.0,0,0.0,0.0,17.0,Active,0,0
3499258,2050782,354892,-7,678150.990,675000,0.0,17473.5,0.0,17473.5,29554.605,...,672446.070,672446.070,0.0,1,0.0,1.0,6.0,Active,0,0
3151685,2331184,454368,-14,0.000,67500,0.0,0.0,0.0,0.0,0.000,...,0.000,0.000,0.0,0,0.0,0.0,61.0,Active,0,0
1171709,1651066,137521,-17,0.000,225000,0.0,6417.0,0.0,6417.0,2250.000,...,0.000,0.000,0.0,1,0.0,1.0,3.0,Active,0,0
3520240,1551326,433073,-25,0.000,90000,0.0,0.0,0.0,0.0,0.000,...,0.000,0.000,0.0,0,0.0,0.0,38.0,Active,0,0
827273,1954894,229130,-14,38628.900,45000,0.0,0.0,0.0,0.0,2250.000,...,38628.900,38628.900,0.0,0,0.0,0.0,4.0,Active,0,0


In [9]:
# data cleaning and feature engineering

# there is one abruptly large value for AMT_PAYMENT_CURRENT
cc_balance['AMT_PAYMENT_CURRENT'][cc_balance['AMT_PAYMENT_CURRENT'] > 4000000] = np.nan

# calculating the total missing values for each previous credit card
cc_balance['MISSING_VALS_TOTAL_CC'] = cc_balance.isna().sum(axis = 1)

# making the MONTHS_BALANCE Positive
cc_balance['MONTHS_BALANCE'] = np.abs(cc_balance['MONTHS_BALANCE'])

# sorting the DataFrame according to the month of status from oldest to latest, for rolling computations
cc_balance = cc_balance.sort_values(by = ['SK_ID_PREV','MONTHS_BALANCE'], ascending = [1,0])

# Creating new features
cc_balance['AMT_DRAWING_SUM'] = cc_balance['AMT_DRAWINGS_ATM_CURRENT'] + cc_balance['AMT_DRAWINGS_CURRENT'] + cc_balance[
                            'AMT_DRAWINGS_OTHER_CURRENT'] + cc_balance['AMT_DRAWINGS_POS_CURRENT']
cc_balance['BALANCE_LIMIT_RATIO'] = cc_balance['AMT_BALANCE'] / (cc_balance['AMT_CREDIT_LIMIT_ACTUAL'] + 0.00001)
cc_balance['CNT_DRAWING_SUM'] = cc_balance['CNT_DRAWINGS_ATM_CURRENT'] + cc_balance['CNT_DRAWINGS_CURRENT'] + cc_balance[
                                    'CNT_DRAWINGS_OTHER_CURRENT'] + cc_balance['CNT_DRAWINGS_POS_CURRENT'] + cc_balance['CNT_INSTALMENT_MATURE_CUM']
cc_balance['MIN_PAYMENT_RATIO'] = cc_balance['AMT_PAYMENT_CURRENT'] / (cc_balance['AMT_INST_MIN_REGULARITY'] + 0.0001)
cc_balance['PAYMENT_MIN_DIFF'] = cc_balance['AMT_PAYMENT_CURRENT'] - cc_balance['AMT_INST_MIN_REGULARITY']
cc_balance['MIN_PAYMENT_TOTAL_RATIO'] = cc_balance['AMT_PAYMENT_TOTAL_CURRENT'] / (cc_balance['AMT_INST_MIN_REGULARITY'] +0.00001)
cc_balance['PAYMENT_MIN_DIFF'] = cc_balance['AMT_PAYMENT_TOTAL_CURRENT'] - cc_balance['AMT_INST_MIN_REGULARITY']
cc_balance['AMT_INTEREST_RECEIVABLE'] = cc_balance['AMT_TOTAL_RECEIVABLE'] - cc_balance['AMT_RECEIVABLE_PRINCIPAL']
cc_balance['SK_DPD_RATIO'] = cc_balance['SK_DPD'] / (cc_balance['SK_DPD_DEF'] + 0.00001)

# calculating the rolling Exponential Weighted Moving Average over months for certain features
rolling_columns = [
    'AMT_BALANCE',
    'AMT_CREDIT_LIMIT_ACTUAL',
    'AMT_RECEIVABLE_PRINCIPAL',
    'AMT_RECIVABLE',
    'AMT_TOTAL_RECEIVABLE',
    'AMT_DRAWING_SUM',
    'BALANCE_LIMIT_RATIO',
    'CNT_DRAWING_SUM',
    'MIN_PAYMENT_RATIO',
    'PAYMENT_MIN_DIFF',
    'MIN_PAYMENT_TOTAL_RATIO',
    'AMT_INTEREST_RECEIVABLE',
    'SK_DPD_RATIO' ]
exp_weighted_columns = ['EXP_' + ele for ele in rolling_columns]
cc_balance[exp_weighted_columns] = cc_balance.groupby(['SK_ID_CURR','SK_ID_PREV'])[rolling_columns].transform(lambda x: x.ewm(alpha = 0.7).mean())


In [11]:
#performing aggregations over SK_ID_PREV

overall_aggregations = {
    'SK_ID_CURR' : ['first'],
    'MONTHS_BALANCE': ['max'],
    'AMT_BALANCE' : ['sum','mean','max'],
    'AMT_CREDIT_LIMIT_ACTUAL' : ['sum','mean','max'],
    'AMT_DRAWINGS_ATM_CURRENT' : ['sum','max'],
    'AMT_DRAWINGS_CURRENT' : ['sum','max'],
    'AMT_DRAWINGS_OTHER_CURRENT' : ['sum','max'],
    'AMT_DRAWINGS_POS_CURRENT' : ['sum','max'],
    'AMT_INST_MIN_REGULARITY' : ['mean','min','max'],
    'AMT_PAYMENT_CURRENT' : ['mean','min','max'],
    'AMT_PAYMENT_TOTAL_CURRENT' : ['mean','min','max'],
    'AMT_RECEIVABLE_PRINCIPAL' : ['sum','mean','max'],
    'AMT_RECIVABLE' : ['sum','mean','max'],
    'AMT_TOTAL_RECEIVABLE' : ['sum','mean','max'],
    'CNT_DRAWINGS_ATM_CURRENT' : ['sum','max'],
    'CNT_DRAWINGS_CURRENT' : ['sum','max'],
    'CNT_DRAWINGS_OTHER_CURRENT' : ['sum','max'],
    'CNT_DRAWINGS_POS_CURRENT' : ['sum','max'],
    'CNT_INSTALMENT_MATURE_CUM' : ['sum','max','min'],
    'SK_DPD' : ['sum','max'],
    'SK_DPD_DEF' : ['sum','max'],

    'AMT_DRAWING_SUM' : ['sum','max'],
    'BALANCE_LIMIT_RATIO' : ['mean','max','min'],
    'CNT_DRAWING_SUM' : ['sum','max'],
    'MIN_PAYMENT_RATIO': ['min','mean'],
    'PAYMENT_MIN_DIFF' : ['min','mean'],
    'MIN_PAYMENT_TOTAL_RATIO' : ['min','mean'], 
    'AMT_INTEREST_RECEIVABLE' : ['min','mean'],
    'SK_DPD_RATIO' : ['max','mean'],

    'EXP_AMT_BALANCE' : ['last'],
    'EXP_AMT_CREDIT_LIMIT_ACTUAL' : ['last'],
    'EXP_AMT_RECEIVABLE_PRINCIPAL' : ['last'],
    'EXP_AMT_RECIVABLE' : ['last'],
    'EXP_AMT_TOTAL_RECEIVABLE' : ['last'],
    'EXP_AMT_DRAWING_SUM' : ['last'],
    'EXP_BALANCE_LIMIT_RATIO' : ['last'],
    'EXP_CNT_DRAWING_SUM' : ['last'],
    'EXP_MIN_PAYMENT_RATIO' : ['last'],
    'EXP_PAYMENT_MIN_DIFF' : ['last'],
    'EXP_MIN_PAYMENT_TOTAL_RATIO' : ['last'],
    'EXP_AMT_INTEREST_RECEIVABLE' : ['last'],
    'EXP_SK_DPD_RATIO' : ['last'],
    'MISSING_VALS_TOTAL_CC' : ['sum']
}
aggregations_for_categories = {
    'SK_DPD' : ['sum','max'],
    'SK_DPD_DEF' : ['sum','max'],
    'BALANCE_LIMIT_RATIO' : ['mean','max','min'],
    'CNT_DRAWING_SUM' : ['sum','max'],
    'MIN_PAYMENT_RATIO': ['min','mean'],
    'PAYMENT_MIN_DIFF' : ['min','mean'],
    'MIN_PAYMENT_TOTAL_RATIO' : ['min','mean'], 
    'AMT_INTEREST_RECEIVABLE' : ['min','mean'],
    'SK_DPD_RATIO' : ['max','mean'],
    'EXP_AMT_DRAWING_SUM' : ['last'],
    'EXP_BALANCE_LIMIT_RATIO' : ['last'],
    'EXP_CNT_DRAWING_SUM' : ['last'],
    'EXP_MIN_PAYMENT_RATIO' : ['last'],
    'EXP_PAYMENT_MIN_DIFF' : ['last'],
    'EXP_MIN_PAYMENT_TOTAL_RATIO' : ['last'],
    'EXP_AMT_INTEREST_RECEIVABLE' : ['last'],
    'EXP_SK_DPD_RATIO' : ['last']
}
aggregations_for_year = {
    'SK_DPD' : ['sum','max'],
    'SK_DPD_DEF' : ['sum','max'],
    'BALANCE_LIMIT_RATIO' : ['mean','max','min'],
    'CNT_DRAWING_SUM' : ['sum','max'],
    'MIN_PAYMENT_RATIO': ['min','mean'],
    'PAYMENT_MIN_DIFF' : ['min','mean'],
    'MIN_PAYMENT_TOTAL_RATIO' : ['min','mean'], 
    'AMT_INTEREST_RECEIVABLE' : ['min','mean'],
    'SK_DPD_RATIO' : ['max','mean'],
    'EXP_AMT_DRAWING_SUM' : ['last'],
    'EXP_BALANCE_LIMIT_RATIO' : ['last'],
    'EXP_CNT_DRAWING_SUM' : ['last'],
    'EXP_MIN_PAYMENT_RATIO' : ['last'],
    'EXP_PAYMENT_MIN_DIFF' : ['last'],
    'EXP_MIN_PAYMENT_TOTAL_RATIO' : ['last'],
    'EXP_AMT_INTEREST_RECEIVABLE' : ['last'],
    'EXP_SK_DPD_RATIO' : ['last']
}

#performing overall aggregations over SK_ID_PREV for all features
cc_balance_aggregated_overall = cc_balance.groupby('SK_ID_PREV').agg(overall_aggregations)
cc_balance_aggregated_overall.columns = ['_'.join(ele).upper() for ele in cc_balance_aggregated_overall.columns]
cc_balance_aggregated_overall.rename(columns = {'SK_ID_CURR_FIRST' : 'SK_ID_CURR'}, inplace = True)

#aggregating over SK_ID_PREV for different categories
contract_status_categories = ['Active','Completed']
cc_balance_aggregated_categories = pd.DataFrame()
for i, contract_type in enumerate(contract_status_categories):
    group = cc_balance[cc_balance['NAME_CONTRACT_STATUS'] == contract_type].groupby('SK_ID_PREV').agg(aggregations_for_categories)
    group.columns = ['_'.join(ele).upper() + '_' + contract_type.upper() for ele in group.columns]
    if i == 0:
        cc_balance_aggregated_categories = group
    else:
        cc_balance_aggregated_categories = cc_balance_aggregated_categories.merge(group, on = 'SK_ID_PREV', how = 'outer')
#aggregating over SK_ID_PREV for rest of the categories
cc_balance_aggregated_categories_rest = cc_balance[(cc_balance['NAME_CONTRACT_STATUS'] != 'Active') & 
                                (cc_balance.NAME_CONTRACT_STATUS != 'Completed')].groupby('SK_ID_PREV').agg(aggregations_for_categories)
cc_balance_aggregated_categories_rest.columns = ['_'.join(ele).upper() + '_REST' for ele in cc_balance_aggregated_categories_rest.columns]
#merging all the categorical aggregations
cc_balance_aggregated_categories = cc_balance_aggregated_categories.merge(cc_balance_aggregated_categories_rest, on = 'SK_ID_PREV', how = 'outer')

#aggregating over SK_ID_PREV for different years
cc_balance['YEAR_BALANCE'] = cc_balance['MONTHS_BALANCE'] //12
cc_balance_aggregated_year = pd.DataFrame()
for year in range(2):
    group = cc_balance[cc_balance['YEAR_BALANCE'] == year].groupby('SK_ID_PREV').agg(aggregations_for_year)
    group.columns = ['_'.join(ele).upper() + '_YEAR_' + str(year) for ele in group.columns]
    if year == 0:
        cc_balance_aggregated_year = group
    else:
        cc_balance_aggregated_year = cc_balance_aggregated_year.merge(group, on = 'SK_ID_PREV', how = 'outer')
#aggregating over SK_ID_PREV for rest of years
cc_balance_aggregated_year_rest = cc_balance[cc_balance['YEAR_BALANCE'] >= 2].groupby('SK_ID_PREV').agg(aggregations_for_year)
cc_balance_aggregated_year_rest.columns = ['_'.join(ele).upper() + '_YEAR_REST' for ele in cc_balance_aggregated_year_rest.columns]
#merging all the yearwise aggregations
cc_balance_aggregated_year = cc_balance_aggregated_year.merge(cc_balance_aggregated_year_rest, on = 'SK_ID_PREV', how = 'outer')
cc_balance = cc_balance.drop('YEAR_BALANCE', axis = 1)

#merging all the aggregations
cc_aggregated = cc_balance_aggregated_overall.merge(cc_balance_aggregated_categories, on = 'SK_ID_PREV', how = 'outer')
cc_aggregated = cc_aggregated.merge(cc_balance_aggregated_year, on = 'SK_ID_PREV', how = 'outer')

#one-hot encoding the categorical column NAME_CONTRACT_STATUS
name_contract_dummies = pd.get_dummies(cc_balance.NAME_CONTRACT_STATUS, prefix='CONTRACT')
contract_names = name_contract_dummies.columns.tolist()     
#merging the one-hot encoded feature with original table
cc_balance = pd.concat([cc_balance, name_contract_dummies], axis=1)
#aggregating over SK_ID_PREV the one-hot encoded columns
aggregated_cc_contract = cc_balance[['SK_ID_PREV'] + contract_names].groupby('SK_ID_PREV').mean()

#merging with the aggregated table
cc_aggregated = cc_aggregated.merge(aggregated_cc_contract, on = 'SK_ID_PREV', how = 'outer')

#now we will aggregate on SK_ID_CURR
#As seen from EDA, since most of the SK_ID_CURR had only 1 credit card, so for aggregations, we will simply take the means
cc_aggregated = cc_aggregated.groupby('SK_ID_CURR', as_index = False).mean()

In [12]:
cc_aggregated

,SK_ID_CURR,MONTHS_BALANCE_MAX,AMT_BALANCE_SUM,AMT_BALANCE_MEAN,AMT_BALANCE_MAX,AMT_CREDIT_LIMIT_ACTUAL_SUM,AMT_CREDIT_LIMIT_ACTUAL_MEAN,AMT_CREDIT_LIMIT_ACTUAL_MAX,AMT_DRAWINGS_ATM_CURRENT_SUM,AMT_DRAWINGS_ATM_CURRENT_MAX,...,EXP_MIN_PAYMENT_TOTAL_RATIO_LAST_YEAR_REST,EXP_AMT_INTEREST_RECEIVABLE_LAST_YEAR_REST,EXP_SK_DPD_RATIO_LAST_YEAR_REST,CONTRACT_Active,CONTRACT_Approved,CONTRACT_Completed,CONTRACT_Demand,CONTRACT_Refused,CONTRACT_Sent proposal,CONTRACT_Signed
0,100006,6.0,0.000,0.000000,0.000,1620000.0,270000.000000,270000.0,0.0,NaN,...,NaN,NaN,NaN,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0
1,100011,75.0,4031676.225,54482.111149,189000.000,12150000.0,164189.189189,180000.0,180000.0,180000.0,...,5.364418e-10,-7.937222e-06,0.000000e+00,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0
2,100013,96.0,1743352.245,18159.919219,161420.220,12645000.0,131718.750000,157500.0,571500.0,157500.0,...,1.632702e-21,-2.582366e-10,2.067998e-24,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0
3,100021,18.0,0.000,0.000000,0.000,11475000.0,675000.000000,675000.0,0.0,NaN,...,NaN,NaN,NaN,0.411765,0.0,0.588235,0.0,0.0,0.0,0.0
4,100023,11.0,0.000,0.000000,0.000,1080000.0,135000.000000,225000.0,0.0,NaN,...,NaN,NaN,NaN,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103553,456244,41.0,5405223.960,131834.730732,453627.675,12150000.0,296341.463415,450000.0,1003500.0,279000.0,...,1.252738e+02,3.213947e+03,0.000000e+00,0.878049,0.0,0.121951,0.0,0.0,0.0,0.0
103554,456246,9.0,105093.855,13136.731875,43490.115,1080000.0,135000.000000,135000.0,0.0,0.0,...,NaN,NaN,NaN,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0
103555,456247,96.0,2205557.640,23216.396211,190202.130,13680000.0,144000.000000,180000.0,202950.0,96750.0,...,4.070538e-21,-5.445355e-09,1.356827e-23,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0
103556,456248,24.0,0.000,0.000000,0.000,20700000.0,900000.000000,900000.0,0.0,NaN,...,0.000000e+00,0.000000e+00,0.000000e+00,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0


In [13]:
cc_aggregated.to_csv("data_preprocessed/cc_aggregated.csv")
# pd.read_csv("data_preprocessed/cc_aggregated.csv", index_col = "SK_ID_CURR")

**application_train and application_test** 

These tables consists of static data relating to the Borrowers. Each row represents one loan application.

1. First we start with cleaning the data by removing the erroneous datapoints. We also remove the rows in train data with categories such that those categories do not appear in test data. We also convert the Region Rating features to categorical becuase we saw from the EDA that they don't follow an ordinal beviour when it comes to Defaulting Characteristics.

2. Inspired from the winner's writeup for the problem, we also predict the missing values of EXT_SOURCE features by building a regression model on the rest of the numeric features.

3. Next we do feature engineering on the numeric features, and generate features based on Domain Knoweldge, such as INCOME TO ANNUITY ratio, EXT_SOURCE means, etc.

4. We also try to predict the interest rates by using the data from the previous applications features, and predicting using the data from application_train features. We also create a feature based on the Target values from application_train where we compute the mean of targets of 500 nearest neighbors of each row.

5. Next we create some features based on the categorical interactions by grouping the data on several categorical combinations and imputing the aggregates for each group as features.

6. We encode the categorical features by response coding, as we didn't want to increase dimensionality by many-folds using OHE.

In [24]:
# load the application_train_test.csv DataFrame.
application_train = pd.read_csv("data/application_train.csv")
application_test = pd.read_csv("data/application_test.csv")


In [25]:
# data cleaning

flag_cols_to_drop = ['FLAG_DOCUMENT_2','FLAG_DOCUMENT_4','FLAG_DOCUMENT_10','FLAG_DOCUMENT_12',
                    'FLAG_DOCUMENT_20']
application_train = application_train.drop(flag_cols_to_drop, axis = 1)
application_test = application_test.drop(flag_cols_to_drop, axis = 1)

# converting age from days to year
application_train['DAYS_BIRTH'] = application_train['DAYS_BIRTH'] * -1 / 365
application_test['DAYS_BIRTH'] = application_test['DAYS_BIRTH'] * -1 / 365

# From the EDA we saw some erroneous values in DAYS_EMPLOYED field
application_train['DAYS_EMPLOYED'][application_train['DAYS_EMPLOYED'] == 365243] = np.nan
application_test['DAYS_EMPLOYED'][application_test['DAYS_EMPLOYED'] == 365243] = np.nan

# OBS Columns have an erroneous value, we'll remove those values
application_train['OBS_30_CNT_SOCIAL_CIRCLE'][application_train['OBS_30_CNT_SOCIAL_CIRCLE'] > 30] == np.nan
application_train['OBS_60_CNT_SOCIAL_CIRCLE'][application_train['OBS_60_CNT_SOCIAL_CIRCLE'] > 30] == np.nan
application_test['OBS_30_CNT_SOCIAL_CIRCLE'][application_test['OBS_30_CNT_SOCIAL_CIRCLE'] > 30] == np.nan
application_test['OBS_60_CNT_SOCIAL_CIRCLE'][application_test['OBS_60_CNT_SOCIAL_CIRCLE'] > 30] == np.nan

# there were also 4 rows with 'XNA' as Gender, removing these rows
application_train = application_train[application_train['CODE_GENDER'] != 'XNA']

# filling the categorical columns with 'XNA' value
categorical_columns = application_train.dtypes[application_train.dtypes == 'object'].index.tolist()
application_train[categorical_columns] = application_train[categorical_columns].fillna('XNA')
application_test[categorical_columns] = application_test[categorical_columns].fillna('XNA')

# converting columns of REGION_RATING_CLIENT to object type, as we saw some complex impact on TARGET variable during EDA
application_train['REGION_RATING_CLIENT'] = application_train['REGION_RATING_CLIENT'].astype('object')
application_train['REGION_RATING_CLIENT_W_CITY'] = application_train['REGION_RATING_CLIENT_W_CITY'].astype('object')
application_test['REGION_RATING_CLIENT'] = application_test['REGION_RATING_CLIENT'].astype('object')
application_test['REGION_RATING_CLIENT_W_CITY'] = application_test['REGION_RATING_CLIENT_W_CITY'].astype('object')

# counting the total NaN values for each application
application_train['MISSING_VALS_TOTAL_APP'] = application_train.isna().sum(axis = 1)
application_test['MISSING_VALS_TOTAL_APP'] = application_test.isna().sum(axis = 1)

In [30]:
#predicting the EXT_SOURCE missing values
#using only numeric columns for predicting the EXT_SOURCES
columns_for_modelling = list(set(application_test.dtypes[application_test.dtypes != 'object'].index.tolist())
                             - set(['EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3','SK_ID_CURR']))


#we'll train an XGB Regression model for predicting missing EXT_SOURCE values
#we will predict in the order of least number of missing value columns to max.
for ext_col in ['EXT_SOURCE_2','EXT_SOURCE_3','EXT_SOURCE_1']:
    #X_model - datapoints which do not have missing values of given column
    #Y_train - values of column trying to predict with non missing values
    #X_train_missing - datapoints in application_train with missing values
    #X_test_missing - datapoints in application_test with missing values
    X_model, X_train_missing, X_test_missing, Y_train = application_train[~application_train[ext_col].isna()][columns_for_modelling], application_train[
                                                        application_train[ext_col].isna()][columns_for_modelling], application_test[
                                                        application_test[ext_col].isna()][columns_for_modelling], application_train[
                                                        ext_col][~application_train[ext_col].isna()]
    xg = XGBRegressor(n_estimators = 1000, max_depth = 3, learning_rate = 0.1, n_jobs = -1, random_state = 59)
    xg.fit(X_model, Y_train)

    application_train[ext_col][application_train[ext_col].isna()] = xg.predict(X_train_missing)
    application_test[ext_col][application_test[ext_col].isna()] = xg.predict(X_test_missing)

    #adding the predicted column to columns for modelling for next column's prediction
    columns_for_modelling = columns_for_modelling + [ext_col]

In [46]:
# feature engineering functions 

def numeric_feature_engineering(data):
    '''
    Function to perform feature engineering on numeric columns based on domain knowledge.
    '''

    # income and credit features
    data['CREDIT_INCOME_RATIO'] = data['AMT_CREDIT'] / (data['AMT_INCOME_TOTAL'] + 0.00001)
    data['CREDIT_ANNUITY_RATIO'] = data['AMT_CREDIT'] / (data['AMT_ANNUITY'] + 0.00001)
    data['ANNUITY_INCOME_RATIO'] = data['AMT_ANNUITY'] / (data['AMT_INCOME_TOTAL'] + 0.00001)
    data['INCOME_ANNUITY_DIFF'] = data['AMT_INCOME_TOTAL'] - data['AMT_ANNUITY']
    data['CREDIT_GOODS_RATIO'] = data['AMT_CREDIT'] / (data['AMT_GOODS_PRICE'] + 0.00001)
    data['CREDIT_GOODS_DIFF'] = data['AMT_CREDIT'] - data['AMT_GOODS_PRICE'] + 0.00001
    data['GOODS_INCOME_RATIO'] = data['AMT_GOODS_PRICE'] / (data['AMT_INCOME_TOTAL'] + 0.00001)
    data['INCOME_EXT_RATIO'] = data['AMT_INCOME_TOTAL'] / (data['EXT_SOURCE_3'] + 0.00001)
    data['CREDIT_EXT_RATIO'] = data['AMT_CREDIT'] / (data['EXT_SOURCE_3'] + 0.00001)
    
    #age ratios and diffs
    data['AGE_EMPLOYED_DIFF'] = data['DAYS_BIRTH'] - data['DAYS_EMPLOYED']
    data['EMPLOYED_TO_AGE_RATIO'] = data['DAYS_EMPLOYED'] / (data['DAYS_BIRTH'] + 0.00001)
    
    #car ratios
    data['CAR_EMPLOYED_DIFF'] = data['OWN_CAR_AGE'] - data['DAYS_EMPLOYED']
    data['CAR_EMPLOYED_RATIO'] = data['OWN_CAR_AGE'] / (data['DAYS_EMPLOYED']+0.00001)
    data['CAR_AGE_DIFF'] = data['DAYS_BIRTH'] - data['OWN_CAR_AGE']
    data['CAR_AGE_RATIO'] = data['OWN_CAR_AGE'] / (data['DAYS_BIRTH'] + 0.00001)
    
    #flag contacts sum
    data['FLAG_CONTACTS_SUM'] = data['FLAG_MOBIL'] + data['FLAG_EMP_PHONE'] + data['FLAG_WORK_PHONE'] + data[
                                'FLAG_CONT_MOBILE'] + data['FLAG_PHONE'] + data['FLAG_EMAIL']

    data['HOUR_PROCESS_CREDIT_MUL'] = data['AMT_CREDIT'] * data['HOUR_APPR_PROCESS_START']
    
    #family members
    data['CNT_NON_CHILDREN'] = data['CNT_FAM_MEMBERS'] - data['CNT_CHILDREN']
    data['CHILDREN_INCOME_RATIO'] = data['CNT_CHILDREN'] / (data['AMT_INCOME_TOTAL'] + 0.00001)
    data['PER_CAPITA_INCOME'] = data['AMT_INCOME_TOTAL'] / (data['CNT_FAM_MEMBERS'] + 1)
    #region ratings
    data['REGIONS_RATING_INCOME_MUL'] = (data['REGION_RATING_CLIENT'] + data['REGION_RATING_CLIENT_W_CITY']) * data['AMT_INCOME_TOTAL'] / 2
    data['REGION_RATING_MAX'] = [max(ele1, ele2) for ele1, ele2 in zip(data['REGION_RATING_CLIENT'], data['REGION_RATING_CLIENT_W_CITY'])]
    data['REGION_RATING_MAX'] = [min(ele1, ele2) for ele1, ele2 in zip(data['REGION_RATING_CLIENT'], data['REGION_RATING_CLIENT_W_CITY'])]
    data['REGION_RATING_MEAN'] = (data['REGION_RATING_CLIENT'] + data['REGION_RATING_CLIENT_W_CITY']) / 2
    data['REGION_RATING_MUL'] = data['REGION_RATING_CLIENT'] * data['REGION_RATING_CLIENT_W_CITY']
    #flag regions
    data['FLAG_REGIONS'] = data['REG_REGION_NOT_LIVE_REGION'] + data['REG_REGION_NOT_WORK_REGION'] + data['LIVE_REGION_NOT_WORK_REGION']+data[
                            'REG_CITY_NOT_LIVE_CITY'] + data['REG_CITY_NOT_WORK_CITY'] + data['LIVE_CITY_NOT_WORK_CITY']   
    #ext_sources
    data['EXT_SOURCE_MEAN'] = (data['EXT_SOURCE_1'] + data['EXT_SOURCE_2'] + data['EXT_SOURCE_3'] ) / 3
    data['EXT_SOURCE_MUL'] = data['EXT_SOURCE_1'] * data['EXT_SOURCE_2'] * data['EXT_SOURCE_3'] 
    data['EXT_SOURCE_MAX'] = [max(ele1,ele2,ele3) for ele1, ele2, ele3 in zip(data['EXT_SOURCE_1'], data['EXT_SOURCE_2'], data['EXT_SOURCE_3'])]
    data['EXT_SOURCE_MIN'] = [min(ele1,ele2,ele3) for ele1, ele2, ele3 in zip(data['EXT_SOURCE_1'], data['EXT_SOURCE_2'], data['EXT_SOURCE_3'])]
    data['EXT_SOURCE_VAR'] = [np.var([ele1,ele2,ele3]) for ele1, ele2, ele3 in zip(data['EXT_SOURCE_1'], data['EXT_SOURCE_2'], data['EXT_SOURCE_3'])]
    data['WEIGHTED_EXT_SOURCE'] =  data.EXT_SOURCE_1 * 2 + data.EXT_SOURCE_2 * 3 + data.EXT_SOURCE_3 * 4
    #apartment scores
    data['APARTMENTS_SUM_AVG'] = data['APARTMENTS_AVG'] + data['BASEMENTAREA_AVG'] + data['YEARS_BEGINEXPLUATATION_AVG'] + data[
                                'YEARS_BUILD_AVG'] + data['COMMONAREA_AVG'] + data['ELEVATORS_AVG'] + data['ENTRANCES_AVG'] + data[
                                'FLOORSMAX_AVG'] + data['FLOORSMIN_AVG'] + data['LANDAREA_AVG'] + data['LIVINGAPARTMENTS_AVG'] + data[
                                'LIVINGAREA_AVG'] + data['NONLIVINGAPARTMENTS_AVG'] + data['NONLIVINGAREA_AVG']

    data['APARTMENTS_SUM_MODE'] = data['APARTMENTS_MODE'] + data['BASEMENTAREA_MODE'] + data['YEARS_BEGINEXPLUATATION_MODE'] + data[
                                'YEARS_BUILD_MODE'] + data['COMMONAREA_MODE'] + data['ELEVATORS_MODE'] + data['ENTRANCES_MODE'] + data[
                                'FLOORSMAX_MODE'] + data['FLOORSMIN_MODE'] + data['LANDAREA_MODE'] + data['LIVINGAPARTMENTS_MODE'] + data[
                                'LIVINGAREA_MODE'] + data['NONLIVINGAPARTMENTS_MODE'] + data['NONLIVINGAREA_MODE'] + data['TOTALAREA_MODE']

    data['APARTMENTS_SUM_MEDI'] = data['APARTMENTS_MEDI'] + data['BASEMENTAREA_MEDI'] + data['YEARS_BEGINEXPLUATATION_MEDI'] + data[
                                'YEARS_BUILD_MEDI'] + data['COMMONAREA_MEDI'] + data['ELEVATORS_MEDI'] + data['ENTRANCES_MEDI'] + data[
                                'FLOORSMAX_MEDI'] + data['FLOORSMIN_MEDI'] + data['LANDAREA_MEDI'] + data['LIVINGAPARTMENTS_MEDI'] + data[
                                'LIVINGAREA_MEDI'] + data['NONLIVINGAPARTMENTS_MEDI'] + data['NONLIVINGAREA_MEDI']
    data['INCOME_APARTMENT_AVG_MUL'] = data['APARTMENTS_SUM_AVG'] * data['AMT_INCOME_TOTAL']
    data['INCOME_APARTMENT_MODE_MUL'] = data['APARTMENTS_SUM_MODE'] * data['AMT_INCOME_TOTAL']
    data['INCOME_APARTMENT_MEDI_MUL'] = data['APARTMENTS_SUM_MEDI'] * data['AMT_INCOME_TOTAL']
    #OBS And DEF
    data['OBS_30_60_SUM'] = data['OBS_30_CNT_SOCIAL_CIRCLE'] + data['OBS_60_CNT_SOCIAL_CIRCLE']
    data['DEF_30_60_SUM'] = data['DEF_30_CNT_SOCIAL_CIRCLE'] + data['DEF_60_CNT_SOCIAL_CIRCLE']
    data['OBS_DEF_30_MUL'] = data['OBS_30_CNT_SOCIAL_CIRCLE'] *  data['DEF_30_CNT_SOCIAL_CIRCLE']
    data['OBS_DEF_60_MUL'] = data['OBS_60_CNT_SOCIAL_CIRCLE'] *  data['DEF_60_CNT_SOCIAL_CIRCLE']
    data['SUM_OBS_DEF_ALL'] = data['OBS_30_CNT_SOCIAL_CIRCLE'] + data['DEF_30_CNT_SOCIAL_CIRCLE'] + data[
                                'OBS_60_CNT_SOCIAL_CIRCLE'] + data['DEF_60_CNT_SOCIAL_CIRCLE']
    data['OBS_30_CREDIT_RATIO'] = data['AMT_CREDIT'] / (data['OBS_30_CNT_SOCIAL_CIRCLE'] + 0.00001)
    data['OBS_60_CREDIT_RATIO'] = data['AMT_CREDIT'] / (data['OBS_60_CNT_SOCIAL_CIRCLE'] + 0.00001)
    data['DEF_30_CREDIT_RATIO'] = data['AMT_CREDIT'] / (data['DEF_30_CNT_SOCIAL_CIRCLE'] + 0.00001)
    data['DEF_60_CREDIT_RATIO'] = data['AMT_CREDIT'] / (data['DEF_60_CNT_SOCIAL_CIRCLE'] + 0.00001)
    #Flag Documents combined
    data['SUM_FLAGS_DOCUMENTS'] = data['FLAG_DOCUMENT_3'] + data['FLAG_DOCUMENT_5'] + data['FLAG_DOCUMENT_6']  + data[
                                'FLAG_DOCUMENT_7'] + data['FLAG_DOCUMENT_8'] + data['FLAG_DOCUMENT_9'] + data[
                                'FLAG_DOCUMENT_11'] + data['FLAG_DOCUMENT_13'] + data['FLAG_DOCUMENT_14'] + data[
                                'FLAG_DOCUMENT_15'] + data['FLAG_DOCUMENT_16'] + data['FLAG_DOCUMENT_17'] + data[
                                'FLAG_DOCUMENT_18'] + data['FLAG_DOCUMENT_19'] + data['FLAG_DOCUMENT_21']
    #details change
    data['DAYS_DETAILS_CHANGE_MUL'] = data['DAYS_LAST_PHONE_CHANGE'] * data['DAYS_REGISTRATION'] * data['DAYS_ID_PUBLISH']
    data['DAYS_DETAILS_CHANGE_SUM'] = data['DAYS_LAST_PHONE_CHANGE'] + data['DAYS_REGISTRATION'] + data['DAYS_ID_PUBLISH']
    #enquires
    data['AMT_ENQ_SUM'] = data['AMT_REQ_CREDIT_BUREAU_HOUR'] + data['AMT_REQ_CREDIT_BUREAU_DAY'] + data['AMT_REQ_CREDIT_BUREAU_WEEK'] + data[
                        'AMT_REQ_CREDIT_BUREAU_MON'] + data['AMT_REQ_CREDIT_BUREAU_QRT'] + data['AMT_REQ_CREDIT_BUREAU_YEAR']
    data['ENQ_CREDIT_RATIO'] = data['AMT_ENQ_SUM'] / (data['AMT_CREDIT'] + 0.00001)

    cnt_payment = cnt_payment_prediction(data)
    data['EXPECTED_CNT_PAYMENT'] = cnt_payment
    data['EXPECTED_INTEREST'] = data['AMT_ANNUITY'] *  data['EXPECTED_CNT_PAYMENT'] - data['AMT_CREDIT']
    data['EXPECTED_INTEREST_SHARE'] = data['EXPECTED_INTEREST'] / (data['AMT_CREDIT'] + 0.00001)
    data['EXPECTED_INTEREST_RATE'] = 2 * 12 * data['EXPECTED_INTEREST'] / (data['AMT_CREDIT'] * (data['EXPECTED_CNT_PAYMENT'] + 1))

    return data

def neighbors_EXT_SOURCE_feature(application_train, application_test):
    '''
    Function to generate a feature which contains the means of TARGET of 500 neighbors of a particular row.
    '''

    #https://www.kaggle.com/c/home-credit-default-risk/discussion/64821
    #imputing the mean of 500 nearest neighbor's target values for each application
    #neighbors are computed using EXT_SOURCE feature and CREDIT_ANNUITY_RATIO

    knn = KNeighborsClassifier(500, n_jobs = -1)

    train_data_for_neighbors = application_train[['EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3','CREDIT_ANNUITY_RATIO']].fillna(0)
    
    train_target = application_train.TARGET
    test_data_for_neighbors = application_test[['EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3','CREDIT_ANNUITY_RATIO']].fillna(0)

    knn.fit(train_data_for_neighbors, train_target)

    train_500_neighbors = knn.kneighbors(train_data_for_neighbors)[1]
    test_500_neighbors = knn.kneighbors(test_data_for_neighbors)[1]

    #adding the means of targets of 500 neighbors to new column
    application_train['TARGET_NEIGHBORS_500_MEAN'] = [application_train['TARGET'].iloc[ele].mean() for ele in train_500_neighbors]
    application_test['TARGET_NEIGHBORS_500_MEAN'] = [application_train['TARGET'].iloc[ele].mean() for ele in test_500_neighbors]
    
    return application_train, application_test
    
def categorical_interaction_features(train_data, test_data):
    '''
    Function to generate some features based on categorical groupings.
    '''

    #now we will create features based on categorical interactions
    columns_to_aggregate_on = [
        ['NAME_CONTRACT_TYPE', 'NAME_INCOME_TYPE', 'OCCUPATION_TYPE'],
        ['CODE_GENDER', 'NAME_FAMILY_STATUS', 'NAME_INCOME_TYPE'],
        ['FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'NAME_INCOME_TYPE'],
        ['NAME_EDUCATION_TYPE','NAME_INCOME_TYPE','OCCUPATION_TYPE'],
        ['OCCUPATION_TYPE','ORGANIZATION_TYPE'],
        ['CODE_GENDER','FLAG_OWN_CAR','FLAG_OWN_REALTY']

    ]
    aggregations = {
        'AMT_ANNUITY' : ['mean','max','min'],
        'ANNUITY_INCOME_RATIO' : ['mean','max','min'],
        'AGE_EMPLOYED_DIFF' : ['mean','min'],
        'AMT_INCOME_TOTAL' : ['mean','max','min'],
        'APARTMENTS_SUM_AVG' : ['mean','max','min'],
        'APARTMENTS_SUM_MEDI' : ['mean','max','min'],
        'EXT_SOURCE_MEAN' : ['mean','max','min'],
        'EXT_SOURCE_1' : ['mean','max','min'],
        'EXT_SOURCE_2' : ['mean','max','min'],
        'EXT_SOURCE_3' : ['mean','max','min']
    }

    #extracting values
    for group in columns_to_aggregate_on:
        #grouping based on categories
        grouped_interactions = train_data.groupby(group).agg(aggregations) 
        grouped_interactions.columns = ['_'.join(ele).upper() + '_AGG_' + '_'.join(group) for ele in grouped_interactions.columns]
        
        #merging with the original data
        train_data = train_data.join(grouped_interactions, on = group)
        test_data = test_data.join(grouped_interactions, on = group)

    return train_data, test_data

def response_fit(data, column):
    '''
    Response Encoding Fit Function
    Function to create a vocabulary with the probability of occurrence of each category for categorical features
    for a given class label.
    '''

    dict_occurrences = {1: {}, 0: {}}
    for label in [0,1]:
        dict_occurrences[label] = dict((data[column][data.TARGET == label].value_counts() / data[column].value_counts()).fillna(0))

    return dict_occurrences

def response_transform(data, column, dict_mapping):
    '''
    Response Encoding Transform Function
    Function to transform the categorical feature into two features, which contain the probability
    of occurrence of that category for each class label.
    '''

    data[column + '_0'] = data[column].map(dict_mapping[0])
    data[column + '_1'] = data[column].map(dict_mapping[1])

def cnt_payment_prediction(data_to_predict):
    '''
    Function to predict the Count_payments on Current Loans using data from previous loans.
    '''

    #https://www.kaggle.com/c/home-credit-default-risk/discussion/64598
    previous_application = pd.read_csv('data/previous_application.csv')
    train_data = previous_application[['AMT_CREDIT', 'AMT_ANNUITY', 'CNT_PAYMENT']].dropna()
    train_data['CREDIT_ANNUITY_RATIO'] = train_data['AMT_CREDIT'] / (train_data['AMT_ANNUITY'] + 1)
    #value to predict is our CNT_PAYMENT
    train_value = train_data.pop('CNT_PAYMENT')

    #test data would be our application_train data
    test_data = data_to_predict[['AMT_CREDIT','AMT_ANNUITY']].fillna(0)
    test_data['CREDIT_ANNUITY_RATIO'] = test_data['AMT_CREDIT'] / (test_data['AMT_ANNUITY'] + 1)

    lgbmr = LGBMRegressor(max_depth = 9, n_estimators = 5000, n_jobs = -1, learning_rate = 0.3, 
                          random_state = 125)
    lgbmr.fit(train_data, train_value)
    
    #predicting the CNT_PAYMENT for test_data
    cnt_payment = lgbmr.predict(test_data)

    return cnt_payment

In [37]:
# doing the feature engineering

# Creating Numeric features based on domain knowledge
application_train = numeric_feature_engineering(application_train)
application_test = numeric_feature_engineering(application_test)


In [40]:
#500 Neighbors Target mean
application_train, application_test = neighbors_EXT_SOURCE_feature(application_train, application_test)


In [44]:
#creating features based on categorical interactions
application_train, application_test = categorical_interaction_features(application_train, application_test)   


In [47]:
#using response coding on categorical features, to keep the dimensionality in check
#categorical columns to perform response coding on
categorical_columns_application = application_train.dtypes[application_train.dtypes == 'object'].index.tolist()
for col in categorical_columns_application:
    #extracting the dictionary with values corresponding to TARGET variable 0 and 1 for each of the categories
    mapping_dictionary = response_fit(application_train, col)
    
    #mapping this dictionary with our DataFrame
    response_transform(application_train, col, mapping_dictionary)
    response_transform(application_test, col, mapping_dictionary)
    #removing the original categorical columns
    _ = application_train.pop(col)
    _ = application_test.pop(col)

In [48]:
application_train

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,WALLSMATERIAL_MODE_0,WALLSMATERIAL_MODE_1,EMERGENCYSTATE_MODE_0,EMERGENCYSTATE_MODE_1,REGIONS_RATING_INCOME_MUL_0,REGIONS_RATING_INCOME_MUL_1,REGION_RATING_MEAN_0,REGION_RATING_MEAN_1,REGION_RATING_MUL_0,REGION_RATING_MUL_1
0,100002,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,25.920548,-637.0,...,0.925941,0.074059,0.930350,0.069650,0.909563,0.090437,0.920995,0.079005,0.920933,0.079067
1,100003,0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,45.931507,-1188.0,...,0.929753,0.070247,0.930350,0.069650,0.919943,0.080057,0.951797,0.048203,0.951797,0.048203
2,100004,0,0,67500.0,135000.0,6750.0,135000.0,0.010032,52.180822,-225.0,...,0.908718,0.091282,0.907392,0.092608,0.928388,0.071612,0.920995,0.079005,0.920933,0.079067
3,100006,0,0,135000.0,312682.5,29686.5,297000.0,0.008019,52.068493,-3039.0,...,0.908718,0.091282,0.907392,0.092608,0.919943,0.080057,0.920995,0.079005,0.920933,0.079067
4,100007,0,0,121500.0,513000.0,21865.5,513000.0,0.028663,54.608219,-3038.0,...,0.908718,0.091282,0.907392,0.092608,0.903079,0.096921,0.920995,0.079005,0.920933,0.079067
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0,0,157500.0,254700.0,27558.0,225000.0,0.032561,25.553425,-236.0,...,0.925941,0.074059,0.930350,0.069650,0.940708,0.059292,0.951797,0.048203,0.951797,0.048203
307507,456252,0,0,72000.0,269550.0,12001.5,225000.0,0.025164,56.917808,NaN,...,0.925941,0.074059,0.930350,0.069650,0.931620,0.068380,0.920995,0.079005,0.920933,0.079067
307508,456253,0,0,153000.0,677664.0,29979.0,585000.0,0.005002,41.002740,-7921.0,...,0.936522,0.063478,0.930350,0.069650,0.922252,0.077748,0.885976,0.114024,0.885976,0.114024
307509,456254,1,0,171000.0,370107.0,20205.0,319500.0,0.005313,32.769863,-4786.0,...,0.925941,0.074059,0.930350,0.069650,0.922072,0.077928,0.920995,0.079005,0.920933,0.079067


In [50]:
# load all the previous table preprocessed
bureau_aggregated = pd.read_csv("data_preprocessed/bureau_merged_aggregated.csv", index_col = "SK_ID_CURR")
previous_aggregated = pd.read_csv("data_preprocessed/previous_application_aggregated.csv", index_col = "SK_ID_CURR")
installments_aggregated = pd.read_csv("data_preprocessed/installments_payments_aggregated.csv", index_col = "SK_ID_CURR")
pos_aggregated = pd.read_csv("data_preprocessed/pos_cash_aggregated_final.csv", index_col = "SK_ID_CURR")
cc_aggregated = pd.read_csv("data_preprocessed/cc_aggregated.csv", index_col = "SK_ID_CURR")

In [51]:
app_train_merged = application_train.merge(bureau_aggregated, on = 'SK_ID_CURR', how = 'left')
app_test_merged = application_test.merge(bureau_aggregated, on = 'SK_ID_CURR', how = 'left')
#merging with aggregated previous_applications
app_train_merged = app_train_merged.merge(previous_aggregated, on = 'SK_ID_CURR', how = 'left')
app_test_merged = app_test_merged.merge(previous_aggregated, on = 'SK_ID_CURR', how = 'left')
#merging with aggregated installments tables
app_train_merged = app_train_merged.merge(installments_aggregated, on = 'SK_ID_CURR', how = 'left')
app_test_merged = app_test_merged.merge(installments_aggregated, on = 'SK_ID_CURR', how = 'left')
#merging with aggregated POS_Cash balance table
app_train_merged = app_train_merged.merge(pos_aggregated, on = 'SK_ID_CURR', how = 'left')
app_test_merged = app_test_merged.merge(pos_aggregated, on = 'SK_ID_CURR', how = 'left')
#merging with aggregated credit card table
app_train_merged = app_train_merged.merge(cc_aggregated, on = 'SK_ID_CURR', how = 'left')
app_test_merged = app_test_merged.merge(cc_aggregated, on = 'SK_ID_CURR', how = 'left')



In [52]:
train_data, test_data = reduce_mem_usage(app_train_merged), reduce_mem_usage(app_test_merged)

----------------------------------------------------------------------------------------------------
Memory usage of dataframe: 3634.10 MB
Memory usage after optimization: 1278.33 MB
Decreased by 64.8%
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Memory usage of dataframe: 575.68 MB
Memory usage after optimization: 208.26 MB
Decreased by 63.8%
----------------------------------------------------------------------------------------------------


In [53]:
train_data

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,EXP_MIN_PAYMENT_TOTAL_RATIO_LAST_YEAR_REST,EXP_AMT_INTEREST_RECEIVABLE_LAST_YEAR_REST,EXP_SK_DPD_RATIO_LAST_YEAR_REST,CONTRACT_Active,CONTRACT_Approved,CONTRACT_Completed,CONTRACT_Demand,CONTRACT_Refused,CONTRACT_Sent proposal,CONTRACT_Signed
0,100002,1,0,202500.0,406597.5,24700.5,351000.0,0.018799,25.921875,-637.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100003,0,0,270000.0,1293502.5,35698.5,1129500.0,0.003542,45.937500,-1188.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100004,0,0,67500.0,135000.0,6750.0,135000.0,0.010033,52.187500,-225.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100006,0,0,135000.0,312682.5,29686.5,297000.0,0.008018,52.062500,-3040.0,...,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,100007,0,0,121500.0,513000.0,21865.5,513000.0,0.028656,54.593750,-3038.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307502,456251,0,0,157500.0,254700.0,27558.0,225000.0,0.032562,25.546875,-236.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307503,456252,0,0,72000.0,269550.0,12001.5,225000.0,0.025162,56.906250,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307504,456253,0,0,153000.0,677664.0,29979.0,585000.0,0.005001,41.000000,-7920.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307505,456254,1,0,171000.0,370107.0,20205.0,319500.0,0.005314,32.781250,-4784.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Features Based on Interaction Among Different Tables**

We will create some more features based on interactions between different tables. For example, we will calculate the Annuity to income ratio for previous applications, similarly we will calculate Credit to income ratios, and several such features.

In [54]:
def create_new_features(data):
    '''
    Function to create few more features after the merging of features, by using the
    interactions between various tables.
    '''
    
    #previous applications columns
    prev_annuity_columns = [ele for ele in previous_aggregated.columns if 'AMT_ANNUITY' in ele]
    for col in prev_annuity_columns:
        data['PREV_' + col + '_INCOME_RATIO'] = data[col] / (data['AMT_INCOME_TOTAL'] + 0.00001)
    prev_goods_columns = [ele for ele in previous_aggregated.columns if 'AMT_GOODS' in ele]
    for col in prev_goods_columns:
        data['PREV_' + col + '_INCOME_RATIO'] = data[col] / (data['AMT_INCOME_TOTAL'] + 0.00001)
  
    #credit_card_balance columns
    cc_amt_principal_cols = [ele for ele in cc_aggregated.columns if 'AMT_RECEIVABLE_PRINCIPAL' in ele]
    for col in cc_amt_principal_cols:
        data['CC_' + col + '_INCOME_RATIO'] = data[col] / (data['AMT_INCOME_TOTAL'] + 0.00001)
    cc_amt_recivable_cols = [ele for ele in cc_aggregated.columns if 'AMT_RECIVABLE' in ele]
    for col in cc_amt_recivable_cols:
        data['CC_' + col + '_INCOME_RATIO'] = data[col] / (data['AMT_INCOME_TOTAL'] + 0.00001)
    cc_amt_total_receivable_cols = [ele for ele in cc_aggregated.columns if 'TOTAL_RECEIVABLE' in ele]
    for col in cc_amt_total_receivable_cols:
        data['CC_' + col + '_INCOME_RATIO'] = data[col] / (data['AMT_INCOME_TOTAL'] + 0.00001)
    
    #installments_payments columns
    installments_payment_cols = [ele for ele in installments_aggregated.columns if 'AMT_PAYMENT' in ele and 'RATIO' not in ele and 'DIFF' not in ele]
    for col in installments_payment_cols:
        data['INSTALLMENTS_' + col + '_INCOME_RATIO'] = data[col] / (data['AMT_INCOME_TOTAL'] + 0.00001)
    #https://www.kaggle.com/c/home-credit-default-risk/discussion/64821
    installments_max_installment = ['AMT_INSTALMENT_MEAN_MAX', 'AMT_INSTALMENT_SUM_MAX']
    for col in installments_max_installment:
        data['INSTALLMENTS_ANNUITY_' + col + '_RATIO'] = data['AMT_ANNUITY'] / (data[col] + 0.00001)
    
    #POS_CASH_balance features have been created in its own dataframe itself

    #bureau and bureau_balance columns
    bureau_days_credit_cols = [ele for ele in bureau_aggregated.columns if 'DAYS_CREDIT' in ele and 'ENDDATE' not in ele and 'UPDATE' not in ele]
    for col in bureau_days_credit_cols:
        data['BUREAU_' + col + '_EMPLOYED_DIFF'] = data[col] - data['DAYS_EMPLOYED']
        data['BUREAU_' + col + '_REGISTRATION_DIFF'] = data[col] - data['DAYS_REGISTRATION']  
    bureau_overdue_cols = [ele for ele in bureau_aggregated.columns if 'AMT_CREDIT' in ele and 'OVERDUE' in ele]
    for col in bureau_overdue_cols:
        data['BUREAU_' + col + '_INCOME_RATIO'] = data[col] / (data['AMT_INCOME_TOTAL'] + 0.00001)
    bureau_amt_annuity_cols = [ele for ele in bureau_aggregated.columns if 'AMT_ANNUITY' in ele and 'CREDIT'  not in ele]
    for col in bureau_amt_annuity_cols:
        data['BUREAU_' + col + '_INCOME_RATIO'] = data[col] / (data['AMT_INCOME_TOTAL'] + 0.00001)

In [56]:
create_new_features(train_data)
create_new_features(test_data)

print("After Pre-processing, aggregation, merging and Feature Engineering,")
print(f"Final Shape of Training Data = {train_data.shape}")
print(f"Final Shape of Test Data = {test_data.shape}")



After Pre-processing, aggregation, merging and Feature Engineering,
Final Shape of Training Data = (307507, 1634)
Final Shape of Test Data = (48744, 1633)


In [59]:
train_data

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,BUREAU_AMT_CREDIT_SUM_OVERDUE_SUM_CREDITACTIVE_CLOSED_INCOME_RATIO,BUREAU_AMT_CREDIT_MAX_OVERDUE_MAX_CREDITACTIVE_ACTIVE_INCOME_RATIO,BUREAU_AMT_CREDIT_MAX_OVERDUE_SUM_CREDITACTIVE_ACTIVE_INCOME_RATIO,BUREAU_AMT_CREDIT_SUM_OVERDUE_MAX_CREDITACTIVE_ACTIVE_INCOME_RATIO,BUREAU_AMT_CREDIT_SUM_OVERDUE_SUM_CREDITACTIVE_ACTIVE_INCOME_RATIO,BUREAU_AMT_CREDIT_MAX_OVERDUE_MAXCREDIT_ACTIVE_REST_INCOME_RATIO,BUREAU_AMT_CREDIT_MAX_OVERDUE_SUMCREDIT_ACTIVE_REST_INCOME_RATIO,BUREAU_AMT_CREDIT_SUM_OVERDUE_MAXCREDIT_ACTIVE_REST_INCOME_RATIO,BUREAU_AMT_CREDIT_SUM_OVERDUE_SUMCREDIT_ACTIVE_REST_INCOME_RATIO,BUREAU_AMT_ANNUITY_MEAN_OVERALL_INCOME_RATIO
0,100002,1,0,202500.0,406597.5,24700.5,351000.0,0.018799,25.921875,-637.0,...,0.0,0.000200,0.000200,0.0,0.0,NaN,NaN,NaN,NaN,0.000000
1,100003,0,0,270000.0,1293502.5,35698.5,1129500.0,0.003542,45.937500,-1188.0,...,0.0,0.000000,0.000000,0.0,0.0,NaN,NaN,NaN,NaN,NaN
2,100004,0,0,67500.0,135000.0,6750.0,135000.0,0.010033,52.187500,-225.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100006,0,0,135000.0,312682.5,29686.5,297000.0,0.008018,52.062500,-3040.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,0,121500.0,513000.0,21865.5,513000.0,0.028656,54.593750,-3038.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307502,456251,0,0,157500.0,254700.0,27558.0,225000.0,0.032562,25.546875,-236.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307503,456252,0,0,72000.0,269550.0,12001.5,225000.0,0.025162,56.906250,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307504,456253,0,0,153000.0,677664.0,29979.0,585000.0,0.005001,41.000000,-7920.0,...,0.0,NaN,0.000000,0.0,0.0,NaN,NaN,NaN,NaN,0.381500
307505,456254,1,0,171000.0,370107.0,20205.0,319500.0,0.005314,32.781250,-4784.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000


In [60]:
train_data.to_csv("data_preprocessed/train_data.csv")
test_data.to_csv("data_preprocessed/test_data.csv")
# pd.read_csv("data_preprocessed/train/test_data.csv", index_col = "SK_ID_CURR")